# Synthetic Data Generation for Datasheet Extraction

An end-to-end pipeline designed to solve the scarcity of labeled training data in the electronics domain. The system procedurally generates high-fidelity synthetic datasheets (HTML/PDF) paired with structured JSON ground truth. It employs a 'Deep & Wide' strategy, covering both complex multi-condition tables (Active components) and standardized specifications (Passive components), to train state-of-the-art Document Understanding models for automated specification analysis.

In [1]:
import ollama
import json
import random
import os
import requests
import re
import datetime
import copy
import uuid
import math
from jinja2 import Template
from IPython.display import display, HTML, JSON, Markdown
from typing import Dict, List, Tuple, Optional, Any

In [2]:
try:
    # --- שלב 2: שליחת בקשה (Prompt) למודל ---
    MODEL_TO_USE = 'qwen2.5:3b' 

    print(f"--- 💬 שולח בקשה למודל: {MODEL_TO_USE} ---")
    
    response = ollama.chat(
        model=MODEL_TO_USE,
        messages=[
            {
                'role': 'user',
                'content': 'כתוב קוד ב C שיוצר מערך של 10 איברים בסדר איברים יורד',
            },
        ]
    )

    print("--- 💡 תשובת המודל ---")
    print(response['message']['content'])

except Exception as e:
    print("\n--- ❌ אירעה שגיאה! ---")
    print(f"פרטי השגיאה: {e}")
    print("\n--- טיפים לפתרון ---")
    print("1. ודא שאפליקציית OLLAMA פועלת ברקע.")
    print(f"2. ודא שהמודל '{MODEL_TO_USE}' באמת מותקן (הר בטרמינל 'ollama list').")
    print("3. אם הוא לא מותקן, הרץ בטרמינל 'ollama pull <model_name>' (למשל 'ollama pull llama3').")

--- 💬 שולח בקשה למודל: qwen2.5:3b ---
--- 💡 תשובת המודל ---
בהצגת C++, הנה קוד להזנת מערך של 10 איברים בסדר אנכי (מתחת למעלה):

```cpp
#include <iostream>
using namespace std;

int main() {
    int array[10];
    
    //initialize the array with decreasing values from index 9 to 0
    for(int i = 0; i < 10; i++) {
        array[i] = 10 - i;
    }

    cout << "The array in descending order is: \n";
    for (int j = 0; j < 10; j++) {
        cout << array[j] << " ";
    }
    
    return 0;
}
```

כעת הנה קוד שestructורcido באופן דומה, אבל ב-C:

```c
#include <stdio.h>

int main() {
    int array[10];
    
    //initialize the array with decreasing values from index 9 to 0
    for(int i = 0; i < 10; i++) {
        array[i] = 10 - i;
    }

    printf("The array in descending order is: \n");
    for (int j = 0; j < 10; j++) {
        printf("%d ", array[j]);
    }
    
    return 0;
}
```

שני קודים שמים את המערך בסדר איברים יורד - מ-9 ל-0.


In [5]:
MODEL_TO_USE = 'qwen2.5:3b' 

# ==========================================
# 1. CONFIGURATION & CONSTANTS
# ==========================================

# E-Series standard values for resistors and capacitors
E_SERIES = {
    "E12": [1.0, 1.2, 1.5, 1.8, 2.2, 2.7, 3.3, 3.9, 4.7, 5.6, 6.8, 8.2],
    "E24": [1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0, 
            3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1],
    "E96": [1.00, 1.02, 1.05, 1.07, 1.10, 1.13, 1.15, 1.18, 1.21, 1.24,
            1.27, 1.30, 1.33, 1.37, 1.40, 1.43, 1.47, 1.50, 1.54, 1.58,
            1.62, 1.65, 1.69, 1.74, 1.78, 1.82, 1.87, 1.91, 1.96, 2.00,
            2.05, 2.10, 2.15, 2.21, 2.26, 2.32, 2.37, 2.43, 2.49, 2.55,
            2.61, 2.67, 2.74, 2.80, 2.87, 2.94, 3.01, 3.09, 3.16, 3.24,
            3.32, 3.40, 3.48, 3.57, 3.65, 3.74, 3.83, 3.92, 4.02, 4.12,
            4.22, 4.32, 4.42, 4.53, 4.64, 4.75, 4.87, 4.99, 5.11, 5.23,
            5.36, 5.49, 5.62, 5.76, 5.90, 6.04, 6.19, 6.34, 6.49, 6.65,
            6.81, 6.98, 7.15, 7.32, 7.50, 7.68, 7.87, 8.06, 8.25, 8.45,
            8.66, 8.87, 9.09, 9.31, 9.53, 9.76]
}

# Standard tolerance values (%)
STANDARD_TOLERANCES = [0.01, 0.05, 0.1, 0.25, 0.5, 1, 2, 5, 10, 20]

# Parameters that should ONLY show Typ (no Min/Max range)
SINGLE_VALUE_PARAMS = {
    "tolerance", "tcr", "temperature_coefficient", "capacitance_tolerance",
    "derating_curve", "max_operating_temp", "tj_max", "package_code", 
    "package_type", "shielding", "core_material"
}

# Tolerance mapping per archetype
TOLERANCE_MAP = {
    "Precision_ThinFilm": 0.001,      # ±0.1%
    "Standard_SMD": 0.05,              # ±5%
    "High_Power_THT": 0.10,            # ±10%
    "Ceramic_MLCC": 0.10,              # ±10%
    "Electrolytic_Alum": 0.20,         # ±20%
    "Tantalum_Solid": 0.10,            # ±10%
    "Signal_Small_Signal": 0.05,       # ±5%
    "Power_Rectifier": 0.05,           # ±5%
    "Schottky_Barrier": 0.05,          # ±5%
    "HV_Power_THT": 0.10,              # ±10%
    "MV_Power_SMD": 0.05,              # ±5%
    "LV_Logic_Level": 0.05,            # ±5%
    "LDO_Low_Voltage_CMOS": 0.02,      # ±2%
    "LDO_High_Voltage_Bipolar": 0.05,  # ±5%
    "Power_Ferrite": 0.20,             # ±20%
    "RF_Ceramic": 0.05,                # ±5%
    "Common_Mode_Choke": 0.20,         # ±20%
    "Small_Signal": 0.10,              # ±10%
    "Power_BJT": 0.20,                 # ±20%
    "General_Purpose": 0.05,           # ±5%
    "Precision_Zero_Drift": 0.01,      # ±1%
    "High_Speed": 0.05                 # ±5%
}

# Process corners for semiconductor variation
PROCESS_CORNERS = {
    "TT": {"min_factor": 0.90, "typ_factor": 1.00, "max_factor": 1.10},  # Typical
    "SS": {"min_factor": 0.80, "typ_factor": 0.90, "max_factor": 1.00},  # Slow
    "FF": {"min_factor": 1.00, "typ_factor": 1.10, "max_factor": 1.20}   # Fast
}

# Temperature derating factors
TEMP_DERATING = {
    25: 1.00,
    70: 0.90,
    85: 0.85,
    100: 0.75,
    125: 0.65,
    150: 0.50,
    175: 0.40
}

# Unit conversion table (base_unit, target_unit) -> factor
UNIT_CONVERSION = {
    # Capacitance
    ("pF", "nF"): 1e-3,
    ("pF", "µF"): 1e-6,
    ("pF", "mF"): 1e-9,
    ("nF", "pF"): 1e3,
    ("nF", "µF"): 1e-3,
    ("µF", "pF"): 1e6,
    ("µF", "nF"): 1e3,
    ("µF", "mF"): 1e-3,
    
    # Current
    ("A", "mA"): 1e3,
    ("A", "µA"): 1e6,
    ("A", "nA"): 1e9,
    ("mA", "A"): 1e-3,
    ("mA", "µA"): 1e3,
    ("µA", "A"): 1e-6,
    ("µA", "mA"): 1e-3,
    
    # Voltage
    ("V", "mV"): 1e3,
    ("V", "µV"): 1e6,
    ("mV", "V"): 1e-3,
    ("mV", "µV"): 1e3,
    
    # Resistance
    ("Ω", "kΩ"): 1e-3,
    ("Ω", "MΩ"): 1e-6,
    ("kΩ", "Ω"): 1e3,
    ("kΩ", "MΩ"): 1e-3,
    ("MΩ", "Ω"): 1e6,
    ("MΩ", "kΩ"): 1e3,
    
    # Inductance
    ("nH", "µH"): 1e-3,
    ("nH", "mH"): 1e-6,
    ("µH", "nH"): 1e3,
    ("µH", "mH"): 1e-3,
    ("mH", "µH"): 1e3,
    
    # Frequency
    ("Hz", "kHz"): 1e-3,
    ("Hz", "MHz"): 1e-6,
    ("Hz", "GHz"): 1e-9,
    ("kHz", "Hz"): 1e3,
    ("kHz", "MHz"): 1e-3,
    ("MHz", "kHz"): 1e3,
    ("MHz", "GHz"): 1e-3,
    ("GHz", "MHz"): 1e3
}
POWER_RATINGS_BY_PACKAGE = {
    # --- SMD Standard ---
    "0201": 0.05,    # 1/20W
    "0402": 0.063,   # 1/16W
    "0603": 0.1,     # 1/10W
    "0805": 0.125,   # 1/8W
    "1206": 0.25,    # 1/4W
    "1210": 0.5,     # 1/2W
    "2010": 0.75,    # 3/4W
    "2512": 1.0,     # 1W
    
    # --- Power Packages (Added) ---
    "SOT-223": 1.0,  # נפוץ מאוד לרכיבי הספק בינוני
    "DPAK": 20.0,    # TO-252
    "D2PAK": 35.0,   # TO-263
    "TO-220": 30.0,  # נע בדרך כלל בין 20W ל-50W
    "TO-247": 100.0, # רכיבי הספק גבוה
    "TO-264": 140.0, # רכיבי ענק
    
    # --- Axial / THT ---
    "Axial_2W": 2.0,
    "Axial_5W": 5.0
}

# Standard voltage ratings by package (Volts)
VOLTAGE_RATINGS_BY_PACKAGE = {
    "0201": 25,
    "0402": 50,
    "0603": 50,
    "0805": 150,
    "1206": 200,
    "1210": 200,
    "2010": 200,
    "2512": 200,
}

# TCR by archetype (ppm/°C)
TCR_BY_ARCHETYPE = {
    "Precision_ThinFilm": 25,
    "Standard_SMD": 100,
    "High_Power_THT": 200,
    "Thick_Film": 200,
}

# Noise levels (dB) - LOWER is better
NOISE_LEVELS = {
    "Precision_ThinFilm": (-50, -40),  # (Typ, Max)
    "Standard_SMD": (-40, -20),
    "High_Power_THT": (-30, -10),
}
# ==========================================
# UNIFIED COMPONENT DATABASE - EXPANDED EDITION
# ==========================================

UNIFIED_COMPONENT_DB = {

    # ==============================================================================
    # 1. RESISTOR
    # ==============================================================================
    "RESISTOR": {
        "archetypes": ["Standard_SMD", "High_Power_THT", "Precision_ThinFilm"],
        
        "ABS_MAX": [
            {
                "key": "power_rating",
                "symbol": "P<sub>70</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Power Dissipation", "aliases": ["Power Rating", "Rated Power", "Dissipation"]},
                "possible_units": ["W", "mW"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "Ambient 70°C",
                        "limits": {
                            "Standard_SMD": [0.063, 0.1, 0.125, 0.25, 0.5, 1.0],
                            "High_Power_THT": [1, 2, 3, 5, 10, 20, 50, 100],
                            "Precision_ThinFilm": [0.063, 0.1, 0.125, 0.25]
                        }
                    }
                ]
            },
            {
                "key": "max_working_voltage",
                "symbol": "V<sub>MWV</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Working Voltage", "aliases": ["Max Working Voltage", "Limiting Element Voltage", "Rated Voltage"]},
                "possible_units": ["V", "kV"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "DC Continuous",
                        "limits": {
                            "Standard_SMD": [25, 50, 75, 150, 200, 400],
                            "High_Power_THT": [250, 350, 500, 750, 1000, 1500, 2500],
                            "Precision_ThinFilm": [25, 50, 75, 100, 150]
                        }
                    }
                ]
            },
            {
                "key": "max_overload_voltage",
                "symbol": "V<sub>OL</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Overload Voltage", "aliases": ["Overload Voltage", "Short-time Overload Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "5 seconds",
                        "limits": {
                            "Standard_SMD": [50, 100, 150, 300, 500],
                            "High_Power_THT": [500, 1000, 2000, 5000],
                            "Precision_ThinFilm": [50, 100, 200, 300]
                        }
                    }
                ]
            },
            {
                "key": "pulse_withstanding",
                "symbol": "P<sub>pulse</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Pulse Withstanding Power", "aliases": ["Peak Pulse Power", "Surge Power Capability"]},
                "possible_units": ["W"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "1ms pulse",
                        "limits": {
                            "Standard_SMD": [0.5, 1, 2, 5],
                            "High_Power_THT": [50, 100, 200, 500],
                            "Precision_ThinFilm": [0.5, 1, 2]
                        }
                    }
                ]
            },
            {
                "key": "max_operating_temp",
                "symbol": "T<sub>max</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Operating Temperature", "aliases": ["Max Operating Temp", "Upper Category Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Continuous",
                        "limits": {
                            "Standard_SMD": [125, 155],
                            "High_Power_THT": [175, 275],
                            "Precision_ThinFilm": [125, 155]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "resistance",
                "symbol": "R",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Resistance", "aliases": ["Resistance Value", "Ohmic Value", "DC Resistance"]},
                "possible_units": ["Ω", "kΩ", "MΩ"],
                "std_unit": "Ω",
                "scenarios": [
                    {
                        "condition": "Standard E-Series",
                        "limits": {
                            "Standard_SMD": [10, 22, 47, 100, 220, 470, 1000, 2200, 4700, 10000, 100000, 1000000],
                            "High_Power_THT": [0.1, 0.47, 1, 2.2, 4.7, 10, 47, 100],
                            "Precision_ThinFilm": [49.9, 100, 249, 499, 1000, 2000, 4990, 10000, 20000]
                        }
                    }
                ]
            },
            {
                "key": "tolerance",
                "symbol": "Tol",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Resistance Tolerance", "aliases": ["Tolerance", "Resistance Accuracy"]},
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "Standard_SMD": [1, 5],
                            "High_Power_THT": [5, 10],
                            "Precision_ThinFilm": [0.01, 0.05, 0.1, 0.5]
                        }
                    }
                ]
            },
            {
                "key": "tcr",
                "symbol": "TCR",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Temperature Coefficient", "aliases": ["Temp. Coefficient", "TCR"]},
                "possible_units": ["ppm/°C"],
                "std_unit": "ppm/°C",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "Standard_SMD": [100, 200],
                            "High_Power_THT": [200, 500, 1000],
                            "Precision_ThinFilm": [2, 5, 10, 25]
                        }
                    }
                ]
            },
            {
                "key": "load_life_stability",
                "symbol": "ΔR/R",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Load Life Stability", "aliases": ["Load Life", "Long-term Drift", "Stability"]},
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "1000h @ Rated Power",
                        "limits": {
                            "Standard_SMD": [1, 2, 5],
                            "High_Power_THT": [3, 5, 10],
                            "Precision_ThinFilm": [0.05, 0.1, 0.25]
                        }
                    }
                ]
            },
            {
                "key": "voltage_coefficient",
                "symbol": "VCR",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Voltage Coefficient", "aliases": ["Voltage Coefficient of Resistance", "VCR"]},
                "possible_units": ["ppm/V"],
                "std_unit": "ppm/V",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "Standard_SMD": [50, 100],
                            "High_Power_THT": [100, 500],
                            "Precision_ThinFilm": [0.1, 1, 5]
                        }
                    }
                ]
            },
            {
                "key": "noise",
                "symbol": "NI",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Current Noise", "aliases": ["Current Noise Index", "Excess Noise"]},
                "possible_units": ["dB"],
                "std_unit": "dB",
                "scenarios": [
                    {
                        "condition": "Per voltage decade",
                        "limits": {
                            "Standard_SMD": [-20, -10, 0],
                            "High_Power_THT": [0, 10],
                            "Precision_ThinFilm": [-40, -35, -30]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance",
                "symbol": "θ<sub>JA</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Thermal Resistance", "aliases": ["Thermal Resistance J-A", "Rth(j-a)"]},
                "possible_units": ["°C/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "Junction to Ambient",
                        "limits": {
                            "Standard_SMD": [200, 150, 100, 50, 25],
                            "High_Power_THT": [50, 25, 15, 10, 5, 2],
                            "Precision_ThinFilm": [200, 150, 100]
                        }
                    }
                ]
            },
            {
                "key": "derating_curve",
                "symbol": "T<sub>derate</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Power Derating Start", "aliases": ["Derating Temperature", "Rated Ambient Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Linear derating",
                        "limits": {
                            "Standard_SMD": [70],
                            "High_Power_THT": [25, 70],
                            "Precision_ThinFilm": [70, 85]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Package Type", "aliases": ["Case Size", "Package Code", "Size Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Standard_SMD": ["0201", "0402", "0603", "0805", "1206", "1210"],
                            "High_Power_THT": ["TO-220", "TO-247", "Axial_2W", "Axial_5W"],
                            "Precision_ThinFilm": ["0402", "0603", "0805"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 2. CAPACITOR
    # ==============================================================================
    "CAPACITOR": {
        "archetypes": ["Ceramic_MLCC", "Electrolytic_Alum", "Tantalum_Solid"],
        
        "ABS_MAX": [
            {
                "key": "rated_voltage",
                "symbol": "V<sub>R</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Rated Voltage", "aliases": ["Rated DC Voltage", "Working Voltage"]},
                "possible_units": ["V", "kV"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Max Operating",
                        "limits": {
                            "Ceramic_MLCC": [6.3, 10, 16, 25, 50, 100, 250, 630, 1000, 2000],
                            "Electrolytic_Alum": [6.3, 10, 16, 25, 35, 50, 63, 100, 200, 400, 450],
                            "Tantalum_Solid": [2.5, 4, 6.3, 10, 16, 20, 25, 35, 50]
                        }
                    }
                ]
            },
            {
                "key": "ripple_current",
                "symbol": "I<sub>ripple</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Rated Ripple Current", "aliases": ["Max Ripple Current", "Allowable Ripple"]},
                "possible_units": ["mA", "A"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "100kHz, 105°C",
                        "limits": {
                            "Ceramic_MLCC": [100, 500, 1000, 2000],
                            "Electrolytic_Alum": [100, 500, 1000, 2000, 5000, 10000],
                            "Tantalum_Solid": [50, 100, 500, 1000]
                        }
                    }
                ]
            },
            {
                "key": "surge_voltage",
                "symbol": "V<sub>S</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Surge Voltage", "aliases": ["Surge Rating", "Max Surge Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "Ceramic_MLCC": [10, 16, 32, 63, 125, 315, 800],
                            "Electrolytic_Alum": [8, 13, 20, 32, 40, 63, 80, 125],
                            "Tantalum_Solid": [3.3, 5.2, 8, 13, 21, 26, 33]
                        }
                    }
                ]
            },
            {
                "key": "reverse_voltage",
                "symbol": "V<sub>R</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Reverse Voltage", "aliases": ["Max Reverse Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "Ceramic_MLCC": [0],
                            "Electrolytic_Alum": [1, 1.5],
                            "Tantalum_Solid": [0.1, 0.5, 1]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "capacitance",
                "symbol": "C",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Nominal Capacitance", "aliases": ["Capacitance Value", "Capacitance"]},
                "possible_units": ["pF", "nF", "µF", "mF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "Standard Test Freq",
                        "limits": {
                            "Ceramic_MLCC": [10, 22, 100, 1000, 10000, 100000, 1000000, 4700000, 10000000],
                            "Electrolytic_Alum": [10000000, 47000000, 100000000, 470000000, 1000000000, 2200000000],
                            "Tantalum_Solid": [1000000, 4700000, 10000000, 47000000, 100000000]
                        }
                    }
                ]
            },
            {
                "key": "capacitance_tolerance",
                "symbol": "Tol",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Capacitance Tolerance", "aliases": ["Tolerance", "Cap. Tolerance"]},
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "Ceramic_MLCC": [5, 10, 20],
                            "Electrolytic_Alum": [20],
                            "Tantalum_Solid": [10, 20]
                        }
                    }
                ]
            },
            {
                "key": "esr",
                "symbol": "ESR",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Equivalent Series Resistance", "aliases": ["ESR", "Series Resistance"]},
                "possible_units": ["mΩ", "Ω"],
                "std_unit": "mΩ",
                "scenarios": [
                    {
                        "condition": "100kHz",
                        "limits": {
                            "Ceramic_MLCC": [2, 5, 10, 50],
                            "Electrolytic_Alum": [50, 100, 250, 500, 2000],
                            "Tantalum_Solid": [50, 100, 500, 1500]
                        }
                    }
                ]
            },
            {
                "key": "dissipation_factor",
                "symbol": "tan δ",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Dissipation Factor", "aliases": ["DF", "Tangent Delta", "tan δ"]},
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "100kHz/120Hz",
                        "limits": {
                            "Ceramic_MLCC": [2.5, 5, 10],
                            "Electrolytic_Alum": [10, 15, 20, 30],
                            "Tantalum_Solid": [6, 10, 15]
                        }
                    }
                ]
            },
            {
                "key": "impedance",
                "symbol": "Z",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Impedance", "aliases": ["Impedance (Z)", "Max Impedance"]},
                "possible_units": ["mΩ", "Ω"],
                "std_unit": "mΩ",
                "scenarios": [
                    {
                        "condition": "100kHz",
                        "limits": {
                            "Ceramic_MLCC": [5, 10, 50, 100],
                            "Electrolytic_Alum": [50, 100, 500, 1000],
                            "Tantalum_Solid": [100, 500, 1000]
                        }
                    }
                ]
            },
            {
                "key": "temperature_coefficient",
                "symbol": "TC",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Temperature Coefficient", "aliases": ["Temp. Characteristics", "TC"]},
                "possible_units": ["ppm/°C", "%"],
                "std_unit": "ppm/°C",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "Ceramic_MLCC": [30, 500, 7500, 15000],
                            "Electrolytic_Alum": [0],
                            "Tantalum_Solid": [0]
                        }
                    }
                ]
            },
            {
                "key": "leakage_current",
                "symbol": "I<sub>leak</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Leakage Current", "aliases": ["DC Leakage Current", "Leakage"]},
                "possible_units": ["µA", "nA"],
                "std_unit": "µA",
                "scenarios": [
                    {
                        "condition": "At rated voltage",
                        "limits": {
                            "Ceramic_MLCC": [0.01, 0.05, 0.1],
                            "Electrolytic_Alum": [3, 10, 50, 100, 500],
                            "Tantalum_Solid": [0.5, 5, 10, 50]
                        }
                    }
                ]
            },
            {
                "key": "insulation_resistance",
                "symbol": "R<sub>ins</sub>",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Insulation Resistance", "aliases": ["Insulation Resistance (IR)"]},
                "possible_units": ["GΩ", "MΩ"],
                "std_unit": "GΩ",
                "scenarios": [
                    {
                        "condition": "At rated voltage",
                        "limits": {
                            "Ceramic_MLCC": [10, 50, 100],
                            "Electrolytic_Alum": [0.001, 0.01],
                            "Tantalum_Solid": [0.1, 1, 10]
                        }
                    }
                ]
            },
            {
                "key": "self_resonant_freq",
                "symbol": "f<sub>res</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Self Resonant Frequency", "aliases": ["SRF", "Resonant Frequency"]},
                "possible_units": ["MHz", "GHz"],
                "std_unit": "MHz",
                "scenarios": [
                    {
                        "condition": "Typical",
                        "limits": {
                            "Ceramic_MLCC": [100, 500, 1000, 5000],
                            "Electrolytic_Alum": [0.1, 0.5, 1, 5],
                            "Tantalum_Solid": [1, 5, 10, 50]
                        }
                    }
                ]
            },
            {
                "key": "rated_lifetime",
                "symbol": "L<sub>rated</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Rated Lifetime", "aliases": ["Load Life", "Endurance", "Life Expectancy"]},
                "possible_units": ["hours"],
                "std_unit": "hours",
                "scenarios": [
                    {
                        "condition": "At rated temp",
                        "limits": {
                            "Ceramic_MLCC": [0],
                            "Electrolytic_Alum": [1000, 2000, 5000, 10000],
                            "Tantalum_Solid": [0]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Case Code", "aliases": ["Case Size", "Package Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Ceramic_MLCC": ["0201", "0402", "0603", "0805", "1206", "1210"],
                            "Electrolytic_Alum": ["Radial_5mm", "Radial_8mm", "Radial_10mm", "SMD_6.3mm"],
                            "Tantalum_Solid": ["A", "B", "C", "D"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 3. DIODE
    # ==============================================================================
    "DIODE": {
        "archetypes": ["Signal_Small_Signal", "Power_Rectifier", "Schottky_Barrier"],
        
        "ABS_MAX": [
            {
                "key": "reverse_voltage",
                "symbol": "V<sub>RRM</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Repetitive Peak Reverse Voltage", "aliases": ["Peak Reverse Voltage", "Max Reverse Voltage"]},
                "possible_units": ["V", "kV"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Tj=25°C",
                        "limits": {
                            "Signal_Small_Signal": [50, 75, 100],
                            "Power_Rectifier": [200, 400, 600, 800, 1000],
                            "Schottky_Barrier": [20, 30, 40, 60, 100]
                        }
                    }
                ]
            },
            {
                "key": "forward_current",
                "symbol": "I<sub>F(AV)</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Average Rectified Current", "aliases": ["Average Forward Current", "Rectified Current"]},
                "possible_units": ["A", "mA"],
                "std_unit": "A",
                "scenarios": [
                    {
                        "condition": "Specified Tc",
                        "limits": {
                            "Signal_Small_Signal": [0.1, 0.2, 0.3],
                            "Power_Rectifier": [1, 2, 5, 10, 20],
                            "Schottky_Barrier": [1, 3, 5, 10, 20]
                        }
                    }
                ]
            },
            {
                "key": "peak_surge_current",
                "symbol": "I<sub>FSM</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Non-Repetitive Peak Surge Current", "aliases": ["Surge Current", "Peak Forward Surge Current"]},
                "possible_units": ["A"],
                "std_unit": "A",
                "scenarios": [
                    {
                        "condition": "8.3ms half-sine",
                        "limits": {
                            "Signal_Small_Signal": [1, 2, 5],
                            "Power_Rectifier": [30, 50, 100, 200, 400],
                            "Schottky_Barrier": [20, 50, 100, 200]
                        }
                    }
                ]
            },
            {
                "key": "power_dissipation",
                "symbol": "P<sub>tot</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Power Dissipation", "aliases": ["Total Power Dissipation", "Power Loss"]},
                "possible_units": ["W", "mW"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "Tc=25°C",
                        "limits": {
                            "Signal_Small_Signal": [0.15, 0.25, 0.5],
                            "Power_Rectifier": [1.5, 3, 5, 10, 50],
                            "Schottky_Barrier": [1, 2, 5, 10]
                        }
                    }
                ]
            },
            {
                "key": "tj_max",
                "symbol": "T<sub>j(max)</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Junction Temperature", "aliases": ["Max Junction Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "Signal_Small_Signal": [150, 175],
                            "Power_Rectifier": [150, 175],
                            "Schottky_Barrier": [125, 150, 175]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "forward_voltage",
                "symbol": "V<sub>F</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Forward Voltage Drop", "aliases": ["Forward Voltage", "Max Forward Voltage"]},
                "possible_units": ["V", "mV"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "IF=Rated",
                        "limits": {
                            "Signal_Small_Signal": [1.0, 1.25],
                            "Power_Rectifier": [1.0, 1.1, 1.2],
                            "Schottky_Barrier": [0.45, 0.55, 0.70, 0.85]
                        }
                    }
                ]
            },
            {
                "key": "reverse_current",
                "symbol": "I<sub>R</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Reverse Leakage Current", "aliases": ["Reverse Current", "Leakage Current"]},
                "possible_units": ["µA", "mA", "nA"],
                "std_unit": "µA",
                "scenarios": [
                    {
                        "condition": "At VR rated",
                        "limits": {
                            "Signal_Small_Signal": [0.005, 0.01, 0.05, 0.1],
                            "Power_Rectifier": [0.5, 5, 10, 50],
                            "Schottky_Barrier": [0.1, 0.5, 1, 10, 100]
                        }
                    }
                ]
            },
            {
                "key": "reverse_recovery_time",
                "symbol": "t<sub>rr</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Reverse Recovery Time", "aliases": ["Recovery Time"]},
                "possible_units": ["ns"],
                "std_unit": "ns",
                "scenarios": [
                    {
                        "condition": "Standard test",
                        "limits": {
                            "Signal_Small_Signal": [4, 10, 50],
                            "Power_Rectifier": [50, 150, 500, 1500],
                            "Schottky_Barrier": [0]
                        }
                    }
                ]
            },
            {
                "key": "junction_capacitance",
                "symbol": "C<sub>J</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Junction Capacitance", "aliases": ["Total Capacitance", "Diode Capacitance"]},
                "possible_units": ["pF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "VR=4V, f=1MHz",
                        "limits": {
                            "Signal_Small_Signal": [1, 2, 5, 10],
                            "Power_Rectifier": [50, 100, 500, 1000],
                            "Schottky_Barrier": [50, 100, 500, 1500]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance_jc",
                "symbol": "θ<sub>JC</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Thermal Resistance Junction to Case", "aliases": ["Rth(j-c)", "Thermal Res. J-C"]},
                "possible_units": ["°C/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Signal_Small_Signal": [300, 200, 100],
                            "Power_Rectifier": [5, 3, 2, 1, 0.5],
                            "Schottky_Barrier": [10, 5, 3, 2]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance_ja",
                "symbol": "θ<sub>JA</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Thermal Resistance Junction to Ambient", "aliases": ["Rth(j-a)", "Thermal Res. J-A"]},
                "possible_units": ["°C/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "Standard PCB",
                        "limits": {
                            "Signal_Small_Signal": [400, 300, 200],
                            "Power_Rectifier": [50, 40, 30, 20],
                            "Schottky_Barrier": [60, 50, 40, 30]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Package", "aliases": ["Case Type", "Package Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Signal_Small_Signal": ["SOD-323", "SOT-23", "SOD-123"],
                            "Power_Rectifier": ["DO-214AC", "TO-220", "SMC", "D2PAK"],
                            "Schottky_Barrier": ["SMA", "SMB", "SMC", "TO-220"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 4. MOSFET
    # ==============================================================================
    "MOSFET": {
        "archetypes": ["HV_Power_THT", "MV_Power_SMD", "LV_Logic_Level"],
        
        "ABS_MAX": [
            {
                "key": "vdss",
                "symbol": "V<sub>DSS</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Drain-Source Voltage", "aliases": ["Drain-Source Breakdown Voltage", "Max VDS"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Tj=25°C",
                        "limits": {
                            "HV_Power_THT": [500, 600, 650, 800, 1000, 1200],
                            "MV_Power_SMD": [40, 60, 80, 100, 150, 200, 250],
                            "LV_Logic_Level": [12, 20, 25, 30]
                        }
                    }
                ]
            },
            {
                "key": "id_cont",
                "symbol": "I<sub>D</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Continuous Drain Current", "aliases": ["Drain Current", "Continuous Current"]},
                "possible_units": ["A"],
                "std_unit": "A",
                "scenarios": [
                    {
                        "condition": "Tc=25°C",
                        "limits": {
                            "HV_Power_THT": [10, 20, 40, 60, 80],
                            "MV_Power_SMD": [20, 50, 80, 100],
                            "LV_Logic_Level": [1, 2, 5, 8, 12]
                        }
                    }
                ]
            },
            {
                "key": "id_pulse",
                "symbol": "I<sub>DM</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Pulsed Drain Current", "aliases": ["Pulsed Current", "Max Pulsed Drain Current"]},
                "possible_units": ["A"],
                "std_unit": "A",
                "scenarios": [
                    {
                        "condition": "10µs pulse",
                        "limits": {
                            "HV_Power_THT": [40, 80, 160, 240, 320],
                            "MV_Power_SMD": [80, 200, 320, 400],
                            "LV_Logic_Level": [4, 8, 20, 32, 48]
                        }
                    }
                ]
            },
            {
                "key": "vgs_max",
                "symbol": "V<sub>GS</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Gate-Source Voltage", "aliases": ["Max Gate-Source Voltage", "VGS Rating"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Continuous",
                        "limits": {
                            "HV_Power_THT": [20, 30],
                            "MV_Power_SMD": [20],
                            "LV_Logic_Level": [8, 12, 20]
                        }
                    }
                ]
            },
            {
                "key": "power_dissipation",
                "symbol": "P<sub>D</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Power Dissipation", "aliases": ["Power Dissipation", "Max Power"]},
                "possible_units": ["W"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "Tc=25°C",
                        "limits": {
                            "HV_Power_THT": [150, 300, 500, 800],
                            "MV_Power_SMD": [50, 100, 200, 300],
                            "LV_Logic_Level": [1, 2, 5, 10]
                        }
                    }
                ]
            },
            {
                "key": "eas",
                "symbol": "E<sub>AS</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Single Pulse Avalanche Energy", "aliases": ["Avalanche Energy", "Max Avalanche Energy"]},
                "possible_units": ["mJ"],
                "std_unit": "mJ",
                "scenarios": [
                    {
                        "condition": "L=0.1mH",
                        "limits": {
                            "HV_Power_THT": [500, 1000, 2000],
                            "MV_Power_SMD": [200, 500, 1000],
                            "LV_Logic_Level": [50, 100, 200]
                        }
                    }
                ]
            },
            {
                "key": "tj_max",
                "symbol": "T<sub>j(max)</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Junction Temperature", "aliases": ["Max Junction Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "HV_Power_THT": [150, 175],
                            "MV_Power_SMD": [150, 175],
                            "LV_Logic_Level": [150, 175]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "rds_on",
                "symbol": "R<sub>DS(on)</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "On-Resistance", "aliases": ["Static Drain-Source On-Resistance", "Drain-Source On-State Resistance"]},
                "possible_units": ["mΩ", "Ω"],
                "std_unit": "mΩ",
                "scenarios": [
                    {
                        "condition": "VGS=10V (or 4.5V for Logic)",
                        "limits": {
                            "HV_Power_THT": [150, 250, 500, 1200],
                            "MV_Power_SMD": [5, 10, 20, 40],
                            "LV_Logic_Level": [5, 10, 15, 30]
                        }
                    }
                ]
            },
            {
                "key": "qg",
                "symbol": "Q<sub>g</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Total Gate Charge", "aliases": ["Gate Charge", "Total Gate Charge"]},
                "possible_units": ["nC"],
                "std_unit": "nC",
                "scenarios": [
                    {
                        "condition": "Standard Test",
                        "limits": {
                            "HV_Power_THT": [50, 100, 150],
                            "MV_Power_SMD": [20, 50, 80],
                            "LV_Logic_Level": [5, 10, 20]
                        }
                    }
                ]
            },
            {
                "key": "vgs_th",
                "symbol": "V<sub>GS(th)</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Gate Threshold Voltage", "aliases": ["Gate Threshold", "Threshold Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "VDS=VGS, Id=250µA",
                        "limits": {
                            "HV_Power_THT": [2, 3, 4, 5],
                            "MV_Power_SMD": [1.5, 2, 2.5, 3],
                            "LV_Logic_Level": [0.5, 0.8, 1, 1.5]
                        }
                    }
                ]
            },
            {
                "key": "idss",
                "symbol": "I<sub>DSS</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Zero Gate Voltage Drain Current", "aliases": ["Drain-Source Leakage Current", "Zero Gate Voltage Drain Current"]},
                "possible_units": ["µA", "nA"],
                "std_unit": "µA",
                "scenarios": [
                    {
                        "condition": "VDS=rated, VGS=0",
                        "limits": {
                            "HV_Power_THT": [1, 10, 100],
                            "MV_Power_SMD": [1, 10, 100],
                            "LV_Logic_Level": [0.001, 0.01, 1]
                        }
                    }
                ]
            },
            {
                "key": "ciss",
                "symbol": "C<sub>iss</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Input Capacitance", "aliases": ["Input Capacitance"]},
                "possible_units": ["pF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "VGS=0, VDS=25V, f=1MHz",
                        "limits": {
                            "HV_Power_THT": [2000, 5000, 10000],
                            "MV_Power_SMD": [1000, 3000, 5000],
                            "LV_Logic_Level": [500, 1000, 2000]
                        }
                    }
                ]
            },
            {
                "key": "coss",
                "symbol": "C<sub>oss</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Output Capacitance", "aliases": ["Output Capacitance"]},
                "possible_units": ["pF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "VGS=0, VDS=25V, f=1MHz",
                        "limits": {
                            "HV_Power_THT": [200, 500, 1000],
                            "MV_Power_SMD": [100, 300, 600],
                            "LV_Logic_Level": [50, 100, 300]
                        }
                    }
                ]
            },
            {
                "key": "crss",
                "symbol": "C<sub>rss</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Reverse Transfer Capacitance", "aliases": ["Reverse Transfer Capacitance"]},
                "possible_units": ["pF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "VGS=0, VDS=25V, f=1MHz",
                        "limits": {
                            "HV_Power_THT": [50, 100, 200],
                            "MV_Power_SMD": [20, 50, 100],
                            "LV_Logic_Level": [10, 20, 50]
                        }
                    }
                ]
            },
            {
                "key": "td_on",
                "symbol": "t<sub>d(on)</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Turn-On Delay Time", "aliases": ["Turn-On Delay Time"]},
                "possible_units": ["ns"],
                "std_unit": "ns",
                "scenarios": [
                    {
                        "condition": "Standard test",
                        "limits": {
                            "HV_Power_THT": [20, 50, 100],
                            "MV_Power_SMD": [10, 20, 50],
                            "LV_Logic_Level": [5, 10, 20]
                        }
                    }
                ]
            },
            {
                "key": "td_off",
                "symbol": "t<sub>d(off)</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Turn-Off Delay Time", "aliases": ["Turn-Off Delay Time"]},
                "possible_units": ["ns"],
                "std_unit": "ns",
                "scenarios": [
                    {
                        "condition": "Standard test",
                        "limits": {
                            "HV_Power_THT": [50, 100, 200],
                            "MV_Power_SMD": [20, 50, 100],
                            "LV_Logic_Level": [10, 20, 40]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance_jc",
                "symbol": "θ<sub>JC</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Thermal Resistance Junction to Case", "aliases": ["Rth(j-c)", "Thermal Res. J-C"]},
                "possible_units": ["°C/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "HV_Power_THT": [0.3, 0.5, 1, 2],
                            "MV_Power_SMD": [0.5, 1, 2, 5],
                            "LV_Logic_Level": [10, 20, 50]
                        }
                    }
                ]
            },
            {
                "key": "vsd",
                "symbol": "V<sub>SD</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Diode Forward Voltage", "aliases": ["Source-Drain Diode Voltage", "Body Diode Forward Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "IS=Id, VGS=0",
                        "limits": {
                            "HV_Power_THT": [1.2, 1.5],
                            "MV_Power_SMD": [0.9, 1.2],
                            "LV_Logic_Level": [0.7, 1.0]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Package Type", "aliases": ["Case Style", "Package Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "HV_Power_THT": ["TO-220", "TO-247", "TO-264"],
                            "MV_Power_SMD": ["DPAK", "D2PAK", "TO-263", "DFN5x6"],
                            "LV_Logic_Level": ["SOT-23", "SOT-223", "TSOP-6"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 5. VOLTAGE REGULATOR
    # ==============================================================================
    "VOLTAGE_REGULATOR": {
        "archetypes": ["LDO_Low_Voltage_CMOS", "LDO_High_Voltage_Bipolar"],
        
        "ABS_MAX": [
            {
                "key": "vin_max",
                "symbol": "V<sub>IN(max)</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Input Voltage", "aliases": ["Input Voltage Max", "Abs Max Input Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Absolute Max",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [5.5, 6.0, 7.0],
                            "LDO_High_Voltage_Bipolar": [20, 30, 40, 60]
                        }
                    }
                ]
            },
            {
                "key": "power_dissipation",
                "symbol": "P<sub>D</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Power Dissipation", "aliases": ["Power Dissipation", "Max Power"]},
                "possible_units": ["W", "mW"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "Ta=25°C",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [0.3, 0.5, 1],
                            "LDO_High_Voltage_Bipolar": [1, 2, 5, 10, 20]
                        }
                    }
                ]
            },
            {
                "key": "tj_max",
                "symbol": "T<sub>j(max)</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Junction Temperature", "aliases": ["Max Junction Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [125, 150],
                            "LDO_High_Voltage_Bipolar": [125, 150]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "output_voltage",
                "symbol": "V<sub>OUT</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Output Voltage", "aliases": ["Output Voltage", "Nominal Output Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Nominal",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [1.2, 1.5, 1.8, 2.5, 3.0, 3.3],
                            "LDO_High_Voltage_Bipolar": [3.3, 5.0, 9.0, 12.0, 15.0, 24.0]
                        }
                    }
                ]
            },
            {
                "key": "iout_max",
                "symbol": "I<sub>OUT</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Rated Output Current", "aliases": ["Output Current", "Max Output Current"]},
                "possible_units": ["mA", "A"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "Nominal",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [150, 300, 500],
                            "LDO_High_Voltage_Bipolar": [500, 1000, 1500, 3000, 5000]
                        }
                    }
                ]
            },
            {
                "key": "dropout_voltage",
                "symbol": "V<sub>DO</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Dropout Voltage", "aliases": ["Dropout Voltage", "Min Input-Output Voltage"]},
                "possible_units": ["mV", "V"],
                "std_unit": "mV",
                "scenarios": [
                    {
                        "condition": "At Iout Max",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [100, 200, 300],
                            "LDO_High_Voltage_Bipolar": [300, 500, 1000, 1500]
                        }
                    }
                ]
            },
            {
                "key": "output_voltage_accuracy",
                "symbol": "ΔV<sub>OUT</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Output Voltage Accuracy", "aliases": ["Output Accuracy", "Output Tolerance"]},
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "25°C",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [0.5, 1, 2],
                            "LDO_High_Voltage_Bipolar": [1, 2, 4]
                        }
                    }
                ]
            },
            {
                "key": "line_regulation",
                "symbol": "ΔV<sub>OUT</sub>/ΔV<sub>IN</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Line Regulation", "aliases": ["Line Regulation"]},
                "possible_units": ["mV/V", "%/V"],
                "std_unit": "mV/V",
                "scenarios": [
                    {
                        "condition": "Vin range",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [0.01, 0.1, 1],
                            "LDO_High_Voltage_Bipolar": [1, 5, 10]
                        }
                    }
                ]
            },
            {
                "key": "load_regulation",
                "symbol": "ΔV<sub>OUT</sub>/ΔI<sub>OUT</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Load Regulation", "aliases": ["Load Regulation"]},
                "possible_units": ["mV", "%"],
                "std_unit": "mV",
                "scenarios": [
                    {
                        "condition": "0mA to Imax",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [5, 10, 20],
                            "LDO_High_Voltage_Bipolar": [10, 50, 100]
                        }
                    }
                ]
            },
            {
                "key": "psrr",
                "symbol": "PSRR",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Power Supply Rejection Ratio", "aliases": ["PSRR", "Ripple Rejection"]},
                "possible_units": ["dB"],
                "std_unit": "dB",
                "scenarios": [
                    {
                        "condition": "100Hz/1kHz",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [50, 60, 70, 80],
                            "LDO_High_Voltage_Bipolar": [40, 50, 60]
                        }
                    }
                ]
            },
            {
                "key": "quiescent_current",
                "symbol": "I<sub>Q</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Quiescent Current", "aliases": ["Quiescent Current", "Ground Current"]},
                "possible_units": ["µA", "mA"],
                "std_unit": "µA",
                "scenarios": [
                    {
                        "condition": "No load",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [1, 5, 50, 100],
                            "LDO_High_Voltage_Bipolar": [500, 1000, 5000]
                        }
                    }
                ]
            },
            {
                "key": "output_noise",
                "symbol": "V<sub>n</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Output Noise Voltage", "aliases": ["Output Noise", "RMS Noise"]},
                "possible_units": ["µVrms"],
                "std_unit": "µVrms",
                "scenarios": [
                    {
                        "condition": "10Hz to 100kHz",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [10, 30, 50, 100],
                            "LDO_High_Voltage_Bipolar": [50, 100, 200]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance_ja",
                "symbol": "θ<sub>JA</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Thermal Resistance Junction to Ambient", "aliases": ["Rth(j-a)", "Thermal Res. J-A"]},
                "possible_units": ["°C/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "Standard PCB",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [150, 200, 300],
                            "LDO_High_Voltage_Bipolar": [40, 60, 80, 100]
                        }
                    }
                ]
            },
            {
                "key": "enable_threshold",
                "symbol": "V<sub>EN</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Enable Threshold Voltage", "aliases": ["Enable Input High", "Enable Threshold"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Logic high",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [0.8, 1.2, 2.0],
                            "LDO_High_Voltage_Bipolar": [2.0, 2.4]
                        }
                    }
                ]
            },
            {
                "key": "current_limit",
                "symbol": "I<sub>LIM</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Current Limit", "aliases": ["Current Limit Threshold", "Short Circuit Current"]},
                "possible_units": ["mA", "A"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "Typ",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": [200, 400, 600],
                            "LDO_High_Voltage_Bipolar": [600, 1200, 1800, 3500, 6000]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Package", "aliases": ["Case", "Package Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "LDO_Low_Voltage_CMOS": ["SOT-23", "SC-70", "DFN1x1"],
                            "LDO_High_Voltage_Bipolar": ["SOT-223", "TO-252", "TO-220", "TO-263"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 6. INDUCTOR
    # ==============================================================================
    "INDUCTOR": {
        "archetypes": ["Power_Ferrite", "RF_Ceramic", "Common_Mode_Choke"],
        
        "ABS_MAX": [
            {
                "key": "rated_current",
                "symbol": "I<sub>R</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Rated Current", "aliases": ["Rated Current", "Max DC Current"]},
                "possible_units": ["mA", "A"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "Temp Rise 40°C",
                        "limits": {
                            "Power_Ferrite": [1000, 2000, 5000, 10000, 20000],
                            "RF_Ceramic": [100, 200, 500],
                            "Common_Mode_Choke": [500, 1000, 5000]
                        }
                    }
                ]
            },
            {
                "key": "saturation_current",
                "symbol": "I<sub>sat</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Saturation Current", "aliases": ["Saturation Current", "Isat"]},
                "possible_units": ["mA", "A"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "30% L drop",
                        "limits": {
                            "Power_Ferrite": [1200, 2500, 6000, 12000, 25000],
                            "RF_Ceramic": [150, 300, 700],
                            "Common_Mode_Choke": [600, 1200, 6000]
                        }
                    }
                ]
            },
            {
                "key": "max_operating_temp",
                "symbol": "T<sub>max</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Operating Temperature", "aliases": ["Max Operating Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "Power_Ferrite": [125, 155],
                            "RF_Ceramic": [125],
                            "Common_Mode_Choke": [125, 155]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "inductance",
                "symbol": "L",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Inductance", "aliases": ["Inductance", "Inductance Value"]},
                "possible_units": ["nH", "µH", "mH"],
                "std_unit": "µH",
                "scenarios": [
                    {
                        "condition": "Test Freq Defined",
                        "limits": {
                            "Power_Ferrite": [1.0, 2.2, 4.7, 10, 22, 47, 100],
                            "RF_Ceramic": [0.001, 0.005, 0.01, 0.047, 0.1],
                            "Common_Mode_Choke": [100, 470, 1000, 4700]
                        }
                    }
                ]
            },
            {
                "key": "tolerance",
                "symbol": "Tol",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Inductance Tolerance", "aliases": ["Tolerance", "Inductance Tolerance"]},
                "possible_units": ["%"],
                "std_unit": "%",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "Power_Ferrite": [10, 20, 30],
                            "RF_Ceramic": [2, 5, 10],
                            "Common_Mode_Choke": [20, 30]
                        }
                    }
                ]
            },
            {
                "key": "dcr",
                "symbol": "DCR",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "DC Resistance", "aliases": ["DC Resistance", "Max DCR"]},
                "possible_units": ["mΩ", "Ω"],
                "std_unit": "mΩ",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "Power_Ferrite": [5, 10, 20, 50, 100],
                            "RF_Ceramic": [200, 500, 1000, 2000],
                            "Common_Mode_Choke": [10, 50, 100]
                        }
                    }
                ]
            },
            {
                "key": "srf",
                "symbol": "f<sub>res</sub>",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Self Resonant Frequency", "aliases": ["Self Resonant Frequency", "SRF"]},
                "possible_units": ["MHz", "GHz"],
                "std_unit": "MHz",
                "scenarios": [
                    {
                        "condition": "Typ",
                        "limits": {
                            "Power_Ferrite": [5, 10, 20, 50],
                            "RF_Ceramic": [500, 1000, 5000, 10000],
                            "Common_Mode_Choke": [10, 50, 100]
                        }
                    }
                ]
            },
            {
                "key": "quality_factor",
                "symbol": "Q",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Quality Factor", "aliases": ["Quality Factor", "Q Factor"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "At test freq",
                        "limits": {
                            "Power_Ferrite": [20, 30, 40],
                            "RF_Ceramic": [40, 60, 80, 100],
                            "Common_Mode_Choke": [10, 20]
                        }
                    }
                ]
            },
            {
                "key": "core_material",
                "symbol": "Material",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Core Material Type", "aliases": ["Core Material"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Power_Ferrite": ["NiZn", "MnZn", "Iron_Powder"],
                            "RF_Ceramic": ["Ceramic", "Air_Core"],
                            "Common_Mode_Choke": ["NiZn", "MnZn"]
                        }
                    }
                ]
            },
            {
                "key": "shielding",
                "symbol": "Shielding",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Magnetic Shielding", "aliases": ["Shielding Type"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Power_Ferrite": ["Shielded", "Semi-Shielded", "Unshielded"],
                            "RF_Ceramic": ["Unshielded"],                            "Common_Mode_Choke": ["Shielded", "Unshielded"]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Size Code", "aliases": ["Package Size", "Case Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Power_Ferrite": ["0603", "0805", "1210", "4040", "6060", "Shielded_SMD"],
                            "RF_Ceramic": ["0201", "0402", "0603"],
                            "Common_Mode_Choke": ["0805", "1206", "Toroid_THT"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 7. BJT
    # ==============================================================================
    "BJT": {
        "archetypes": ["Small_Signal", "Power_BJT"],
        
        "ABS_MAX": [
            {
                "key": "vceo",
                "symbol": "V<sub>CEO</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Collector-Emitter Voltage", "aliases": ["Collector-Emitter Voltage", "Max VCEO"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Open Base",
                        "limits": {
                            "Small_Signal": [30, 45, 60, 80],
                            "Power_BJT": [60, 80, 100, 140, 250, 400]
                        }
                    }
                ]
            },
            {
                "key": "vcbo",
                "symbol": "V<sub>CBO</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Collector-Base Voltage", "aliases": ["Collector-Base Voltage", "Max VCBO"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Open Emitter",
                        "limits": {
                            "Small_Signal": [40, 60, 80, 100],
                            "Power_BJT": [80, 100, 120, 160, 300, 450]
                        }
                    }
                ]
            },
            {
                "key": "ic_max",
                "symbol": "I<sub>C</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Collector Current", "aliases": ["Collector Current", "Max Collector Current"]},
                "possible_units": ["mA", "A"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "Continuous",
                        "limits": {
                            "Small_Signal": [100, 200, 500, 800],
                            "Power_BJT": [1000, 2000, 5000, 10000, 15000]
                        }
                    }
                ]
            },
            {
                "key": "power_dissipation",
                "symbol": "P<sub>D</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Power Dissipation", "aliases": ["Total Power Dissipation", "Max Power"]},
                "possible_units": ["W", "mW"],
                "std_unit": "W",
                "scenarios": [
                    {
                        "condition": "Tc=25°C",
                        "limits": {
                            "Small_Signal": [0.2, 0.35, 0.5, 1],
                            "Power_BJT": [2, 5, 10, 25, 50, 100]
                        }
                    }
                ]
            },
            {
                "key": "tj_max",
                "symbol": "T<sub>j(max)</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Junction Temperature", "aliases": ["Max Junction Temp"]},
                "possible_units": ["°C"],
                "std_unit": "°C",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "Small_Signal": [150, 175],
                            "Power_BJT": [150, 175, 200]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "hfe",
                "symbol": "h<sub>FE</sub>",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "DC Current Gain", "aliases": ["DC Current Gain", "hFE"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "VCE=5V/10V",
                        "limits": {
                            "Small_Signal": [100, 200, 300, 400],
                            "Power_BJT": [20, 40, 60, 100]
                        }
                    }
                ]
            },
            {
                "key": "vce_sat",
                "symbol": "V<sub>CE(sat)</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Saturation Voltage", "aliases": ["Collector-Emitter Saturation Voltage", "Saturation Voltage"]},
                "possible_units": ["mV", "V"],
                "std_unit": "mV",
                "scenarios": [
                    {
                        "condition": "Ic/Ib=10",
                        "limits": {
                            "Small_Signal": [200, 300, 500],
                            "Power_BJT": [500, 1000, 1500, 2500]
                        }
                    }
                ]
            },
            {
                "key": "vbe_on",
                "symbol": "V<sub>BE(on)</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Base-Emitter Voltage", "aliases": ["Base-Emitter On Voltage", "VBE On"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "At rated Ic",
                        "limits": {
                            "Small_Signal": [0.6, 0.65, 0.7, 0.75],
                            "Power_BJT": [0.7, 0.8, 0.9, 1.0]
                        }
                    }
                ]
            },
            {
                "key": "ft",
                "symbol": "f<sub>T</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Transition Frequency", "aliases": ["Transition Frequency", "Gain Bandwidth Product"]},
                "possible_units": ["MHz", "GHz"],
                "std_unit": "MHz",
                "scenarios": [
                    {
                        "condition": "Typ",
                        "limits": {
                            "Small_Signal": [100, 200, 300, 500],
                            "Power_BJT": [1, 3, 10, 30]
                        }
                    }
                ]
            },
            {
                "key": "cob",
                "symbol": "C<sub>ob</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Output Capacitance", "aliases": ["Output Capacitance", "Collector Output Capacitance"]},
                "possible_units": ["pF"],
                "std_unit": "pF",
                "scenarios": [
                    {
                        "condition": "VCB=10V",
                        "limits": {
                            "Small_Signal": [2, 5, 10, 20],
                            "Power_BJT": [50, 100, 500, 1000]
                        }
                    }
                ]
            },
            {
                "key": "thermal_resistance_jc",
                "symbol": "θ<sub>JC</sub>",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Thermal Resistance Junction to Case", "aliases": ["Rth(j-c)", "Thermal Res. J-C"]},
                "possible_units": ["°C/W"],
                "std_unit": "°C/W",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Small_Signal": [200, 150, 100, 50],
                            "Power_BJT": [5, 3, 2, 1, 0.7]
                        }
                    }
                ]
            },
            {
                "key": "icbo",
                "symbol": "I<sub>CBO</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Collector-Base Leakage Current", "aliases": ["Collector Cutoff Current", "Leakage Current"]},
                "possible_units": ["nA", "µA"],
                "std_unit": "nA",
                "scenarios": [
                    {
                        "condition": "VCB=rated",
                        "limits": {
                            "Small_Signal": [10, 50, 100],
                            "Power_BJT": [100, 500, 1000, 5000]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Package", "aliases": ["Case", "Package Code"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "Small_Signal": ["SOT-23", "TO-92", "SOT-323"],
                            "Power_BJT": ["TO-126", "TO-220", "TO-247", "DPAK", "SOT-223"]
                        }
                    }
                ]
            }
        ]
    },

    # ==============================================================================
    # 8. OPAMP
    # ==============================================================================
    "OPAMP": {
        "archetypes": ["General_Purpose", "Precision_Zero_Drift", "High_Speed"],
        
        "ABS_MAX": [
            {
                "key": "supply_voltage",
                "symbol": "V<sub>S</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Total Supply Voltage", "aliases": ["Supply Voltage", "Max Supply Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Total (V+ - V-)",
                        "limits": {
                            "General_Purpose": [16, 32, 36, 44],
                            "Precision_Zero_Drift": [5.5, 12, 16],
                            "High_Speed": [5.5, 12]
                        }
                    }
                ]
            },
            {
                "key": "differential_input_voltage",
                "symbol": "V<sub>ID</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Differential Input Voltage", "aliases": ["Max Differential Input", "Diff. Input Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "Max",
                        "limits": {
                            "General_Purpose": [32, 36, 44],
                            "Precision_Zero_Drift": [5.5, 12],
                            "High_Speed": [5.5, 12]
                        }
                    }
                ]
            },
            {
                "key": "power_dissipation",
                "symbol": "P<sub>D</sub>",
                "spec_type": "max_rating",
                "llm_context": {"formal_name": "Maximum Power Dissipation", "aliases": ["Power Dissipation", "Max Power"]},
                "possible_units": ["mW", "W"],
                "std_unit": "mW",
                "scenarios": [
                    {
                        "condition": "Ta=25°C",
                        "limits": {
                            "General_Purpose": [500, 1000],
                            "Precision_Zero_Drift": [300, 500],
                            "High_Speed": [500, 1000]
                        }
                    }
                ]
            }
        ],
        
        "ELEC_CHAR": [
            {
                "key": "gbw",
                "symbol": "GBW",
                "spec_type": "nominal",
                "llm_context": {"formal_name": "Gain Bandwidth Product", "aliases": ["Gain Bandwidth", "GBW"]},
                "possible_units": ["kHz", "MHz"],
                "std_unit": "MHz",
                "scenarios": [
                    {
                        "condition": "Standard",
                        "limits": {
                            "General_Purpose": [1, 3, 10],
                            "Precision_Zero_Drift": [0.5, 1, 5],
                            "High_Speed": [50, 100, 500, 1000]
                        }
                    }
                ]
            },
            {
                "key": "offset_voltage",
                "symbol": "V<sub>os</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Input Offset Voltage", "aliases": ["Input Offset Voltage", "Vos"]},
                "possible_units": ["µV", "mV"],
                "std_unit": "mV",
                "scenarios": [
                    {
                        "condition": "25°C",
                        "limits": {
                            "General_Purpose": [2, 5, 7],
                            "Precision_Zero_Drift": [0.005, 0.01, 0.05],
                            "High_Speed": [1, 3, 5]
                        }
                    }
                ]
            },
            {
                "key": "offset_drift",
                "symbol": "ΔV<sub>os</sub>/ΔT",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Input Offset Voltage Drift", "aliases": ["Offset Voltage Drift", "TCV"]},
                "possible_units": ["µV/°C", "nV/°C"],
                "std_unit": "µV/°C",
                "scenarios": [
                    {
                        "condition": "Typ",
                        "limits": {
                            "General_Purpose": [5, 10, 20],
                            "Precision_Zero_Drift": [0.005, 0.01, 0.05],
                            "High_Speed": [2, 5, 10]
                        }
                    }
                ]
            },
            {
                "key": "input_bias_current",
                "symbol": "I<sub>b</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Input Bias Current", "aliases": ["Input Bias Current", "Bias Current"]},
                "possible_units": ["pA", "nA", "µA"],
                "std_unit": "nA",
                "scenarios": [
                    {
                        "condition": "25°C",
                        "limits": {
                            "General_Purpose": [20, 50, 100, 500],
                            "Precision_Zero_Drift": [0.001, 0.01, 1],
                            "High_Speed": [1, 10, 100]
                        }
                    }
                ]
            },
            {
                "key": "input_offset_current",
                "symbol": "I<sub>os</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Input Offset Current", "aliases": ["Input Offset Current", "Offset Current"]},
                "possible_units": ["pA", "nA"],
                "std_unit": "nA",
                "scenarios": [
                    {
                        "condition": "25°C",
                        "limits": {
                            "General_Purpose": [5, 20, 50],
                            "Precision_Zero_Drift": [0.001, 0.005, 0.5],
                            "High_Speed": [1, 5, 20]
                        }
                    }
                ]
            },
            {
                "key": "cmrr",
                "symbol": "CMRR",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Common Mode Rejection Ratio", "aliases": ["CMRR", "Common Mode Rejection"]},
                "possible_units": ["dB"],
                "std_unit": "dB",
                "scenarios": [
                    {
                        "condition": "DC",
                        "limits": {
                            "General_Purpose": [70, 80, 100],
                            "Precision_Zero_Drift": [100, 120, 140],
                            "High_Speed": [70, 80, 90]
                        }
                    }
                ]
            },
            {
                "key": "psrr",
                "symbol": "PSRR",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Power Supply Rejection Ratio", "aliases": ["PSRR", "Supply Rejection Ratio"]},
                "possible_units": ["dB"],
                "std_unit": "dB",
                "scenarios": [
                    {
                        "condition": "DC",
                        "limits": {
                            "General_Purpose": [70, 80, 100],
                            "Precision_Zero_Drift": [100, 120, 140],
                            "High_Speed": [60, 70, 80]
                        }
                    }
                ]
            },
            {
                "key": "slew_rate",
                "symbol": "SR",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Slew Rate", "aliases": ["Slew Rate"]},
                "possible_units": ["V/µs"],
                "std_unit": "V/µs",
                "scenarios": [
                    {
                        "condition": "Unity Gain",
                        "limits": {
                            "General_Purpose": [0.5, 1, 3, 10],
                            "Precision_Zero_Drift": [0.1, 0.5, 1],
                            "High_Speed": [50, 100, 500, 1000]
                        }
                    }
                ]
            },
            {
                "key": "supply_current",
                "symbol": "I<sub>S</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Supply Current per Amplifier", "aliases": ["Supply Current", "Quiescent Current per Amp"]},
                "possible_units": ["µA", "mA"],
                "std_unit": "mA",
                "scenarios": [
                    {
                        "condition": "No load",
                        "limits": {
                            "General_Purpose": [0.3, 1, 3, 5],
                            "Precision_Zero_Drift": [0.5, 1, 2],
                            "High_Speed": [5, 10, 20]
                        }
                    }
                ]
            },
            {
                "key": "input_voltage_noise",
                "symbol": "e<sub>n</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Input Voltage Noise Density", "aliases": ["Input Voltage Noise", "Voltage Noise Density"]},
                "possible_units": ["nV/√Hz"],
                "std_unit": "nV/√Hz",
                "scenarios": [
                    {
                        "condition": "1kHz",
                        "limits": {
                            "General_Purpose": [10, 20, 40],
                            "Precision_Zero_Drift": [5, 10, 20],
                            "High_Speed": [5, 10, 20]
                        }
                    }
                ]
            },
            {
                "key": "input_current_noise",
                "symbol": "i<sub>n</sub>",
                "spec_type": "max_limit",
                "llm_context": {"formal_name": "Input Current Noise Density", "aliases": ["Input Current Noise", "Current Noise Density"]},
                "possible_units": ["fA/√Hz", "pA/√Hz"],
                "std_unit": "fA/√Hz",
                "scenarios": [
                    {
                        "condition": "1kHz",
                        "limits": {
                            "General_Purpose": [100, 500, 1000],
                            "Precision_Zero_Drift": [10, 50, 100],
                            "High_Speed": [1, 10, 100]
                        }
                    }
                ]
            },
            {
                "key": "output_voltage_swing",
                "symbol": "V<sub>OUT</sub>",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Output Voltage Swing", "aliases": ["Output Swing", "Max Output Voltage"]},
                "possible_units": ["V"],
                "std_unit": "V",
                "scenarios": [
                    {
                        "condition": "RL=2kΩ",
                        "limits": {
                            "General_Purpose": [12, 13, 14],
                            "Precision_Zero_Drift": [3.5, 4.5, 5],
                            "High_Speed": [3.5, 4.5]
                        }
                    }
                ]
            },
            {
                "key": "open_loop_gain",
                "symbol": "A<sub>OL</sub>",
                "spec_type": "min_limit",
                "llm_context": {"formal_name": "Open Loop Voltage Gain", "aliases": ["Open Loop Gain", "Large Signal Voltage Gain"]},
                "possible_units": ["dB", "V/mV"],
                "std_unit": "dB",
                "scenarios": [
                    {
                        "condition": "DC",
                        "limits": {
                            "General_Purpose": [90, 100, 110],
                            "Precision_Zero_Drift": [120, 130, 140],
                            "High_Speed": [80, 90, 100]
                        }
                    }
                ]
            }
        ],
        
        "PACKAGE": [
            {
                "key": "package_code",
                "symbol": "PKG",
                "spec_type": "mechanical",
                "llm_context": {"formal_name": "Package", "aliases": ["Case Style", "Package Type"]},
                "possible_units": [""],
                "std_unit": "",
                "scenarios": [
                    {
                        "condition": "N/A",
                        "limits": {
                            "General_Purpose": ["SOIC-8", "TSSOP-8", "SOT-23-5"],
                            "Precision_Zero_Drift": ["SOIC-8", "MSOP-8"],
                            "High_Speed": ["SOT-23-5", "SC-70", "QFN"]
                        }
                    }
                ]
            }
        ]
    }
}

# Phase 1 Implementation: Synthetic Generation Pipeline Analysis

In [6]:

# ==========================================
# 2. PHYSICAL CORRELATION ENGINE
# ==========================================
class PhysicalCorrelationEngine:
    """
    Ensures physically realistic relationships between parameters.
    Example: High VDS → High RDS(on), Low Capacitance → High SRF
    """
    
    @staticmethod
    def adjust_rds_based_on_vds(vds_value: float, base_rds: float) -> float:
        """MOSFETs: Higher voltage rating → Higher on-resistance"""
        if vds_value > 500:
            return base_rds * random.uniform(2.0, 5.0)
        elif vds_value > 200:
            return base_rds * random.uniform(1.5, 3.0)
        elif vds_value > 100:
            return base_rds * random.uniform(1.0, 2.0)
        else:
            return base_rds * random.uniform(0.5, 1.2)
    
    @staticmethod
    def adjust_forward_current_based_on_package(package: str, base_current: float) -> float:
        """Diodes: Larger packages → Higher current capability"""
        if any(p in package for p in ["TO-220", "TO-247", "D2PAK"]):
            return base_current * random.uniform(1.5, 3.0)
        elif any(p in package for p in ["SMC", "DO-214"]):
            return base_current * random.uniform(1.0, 1.5)
        else:
            return base_current * random.uniform(0.5, 1.0)
            
    @staticmethod        
    def adjust_capacitance_vs_voltage(cap_base, voltage_rating, archetype):
        if "Electrolytic" in archetype and voltage_rating > 50:
            return cap_base * random.uniform(0.8, 0.95)
        return cap_base
    
    @staticmethod
    def adjust_esr_based_on_capacitance(cap_value: float, base_esr: float) -> float:
        """Capacitors: Higher capacitance → Lower ESR (usually)"""
        if cap_value > 100e6:  # >100µF
            return base_esr * random.uniform(0.3, 0.7)
        elif cap_value > 10e6:  # >10µF
            return base_esr * random.uniform(0.6, 1.0)
        else:
            return base_esr * random.uniform(1.0, 2.0)
    
    @staticmethod
    def adjust_power_dissipation_based_on_package(package: str, base_power: float) -> float:
        """Larger packages → Better thermal performance"""
        thermal_map = {
            "TO-220": 2.5, "TO-247": 3.5, "TO-264": 4.0,
            "DPAK": 1.5, "D2PAK": 2.0,
            "SOT-23": 0.3, "SOT-223": 0.8,
            "0805": 0.2, "1206": 0.4, "2512": 1.0
        }
        for pkg, factor in thermal_map.items():
            if pkg in package:
                return base_power * factor
        return base_power

# ==========================================
# 3. HELPER FUNCTIONS
# ==========================================
def apply_final_rounding_atomic(
    values_triple: Tuple[Optional[float], Optional[float], Optional[float]], 
    tolerance: float,
    unit: str,
    param_key: str = ""
) -> Tuple[Optional[float], Optional[float], Optional[float]]:
    """
    🆕 ATOMIC ROUNDING: Round Min/Typ/Max together while preserving tolerance
    
    This function ensures that after rounding:
    1. Min ≤ Typ ≤ Max (ordering preserved)
    2. (Max - Typ) / Typ ≈ tolerance (tolerance preserved)
    3. (Typ - Min) / Typ ≈ tolerance (symmetry preserved)
    
    Args:
        values_triple: (min, typ, max) tuple
        tolerance: Expected tolerance (e.g., 0.05 for 5%)
        unit: Unit of measurement
        param_key: Parameter key for special handling
    
    Returns:
        Tuple of rounded (min, typ, max) values
    """
    raw_min, raw_typ, raw_max = values_triple

    # --- תיקון קריטי: אם אין Typ, אי אפשר לבצע עיגול אטומי מבוסס Typ ---
    if not isinstance(raw_typ, (int, float)):
        return (
            apply_final_rounding(raw_min, "adaptive", unit, param_key),
            apply_final_rounding(raw_typ, "adaptive", unit, param_key),
            apply_final_rounding(raw_max, "adaptive", unit, param_key)
        )
        
    # If any value is None or not numeric, skip atomic rounding
    if not all(isinstance(v, (int, float)) for v in [raw_min, raw_typ, raw_max] if v is not None):
        return (
            apply_final_rounding(raw_min, "adaptive", unit, param_key) if raw_min is not None else None,
            apply_final_rounding(raw_typ, "adaptive", unit, param_key) if raw_typ is not None else None,
            apply_final_rounding(raw_max, "adaptive", unit, param_key) if raw_max is not None else None
        )
    
    # Round Typ first (it's the anchor)
    rounded_typ = apply_final_rounding(raw_typ, "adaptive", unit, param_key)
    
    if rounded_typ == 0:
        return (raw_min, rounded_typ, raw_max)
    
    # Calculate Min/Max from tolerance (ensures perfect consistency)
    rounded_min = rounded_typ * (1 - tolerance)
    rounded_max = rounded_typ * (1 + tolerance)
    
    # Apply appropriate rounding to Min/Max while preserving relative error
    rounded_min = apply_final_rounding(rounded_min, "adaptive", unit, param_key)
    rounded_max = apply_final_rounding(rounded_max, "adaptive", unit, param_key)
    
    # Final check: ensure Min < Typ < Max (handle edge cases)
    if rounded_min >= rounded_typ:
        rounded_min = rounded_typ * 0.99  # Fallback: 1% below
    if rounded_max <= rounded_typ:
        rounded_max = rounded_typ * 1.01  # Fallback: 1% above
    
    return (rounded_min, rounded_typ, rounded_max)
def extract_temperature_from_condition(condition_str: str) -> Optional[int]:
    """Extract temperature value from condition string"""
    import re
    match = re.search(r'(\d+)\s*°C', condition_str)
    if match:
        return int(match.group(1))
    match = re.search(r'T[acj]\s*=\s*(\d+)', condition_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return None

def apply_temperature_derating(base_value: float, temp: int) -> float:
    """Apply derating based on temperature"""
    closest_temp = min(TEMP_DERATING.keys(), key=lambda x: abs(x - temp))
    factor = TEMP_DERATING[closest_temp]
    return base_value * factor

def snap_to_e_series(value: float, series: str = "E24") -> float:
    """Snap a value to nearest E-series standard value"""
    if value <= 0:
        return value
    
    # Get mantissa and exponent
    exponent = 0
    mantissa = value
    
    while mantissa >= 10:
        mantissa /= 10
        exponent += 1
    
    while mantissa < 1:
        mantissa *= 10
        exponent -= 1
    
    # Find nearest E-series value
    e_values = E_SERIES.get(series, E_SERIES["E24"])
    nearest = min(e_values, key=lambda x: abs(x - mantissa))
    
    # Reconstruct value
    return nearest * (10 ** exponent)

def snap_to_standard_tolerance(value: float) -> float:
    """Snap tolerance value to standard values"""
    return min(STANDARD_TOLERANCES, key=lambda x: abs(x - value))

def apply_final_rounding(value: Any, precision_level: str = "adaptive", 
                        unit: str = "", param_key: str = "") -> Any:
    """
    FIXED VERSION v2.1: Smarter rounding that preserves tolerance information
    
    Key changes:
    - Less aggressive rounding for 100-1000 range
    - Context-aware precision for resistance/capacitance
    - Preserves boundary information near E-series values
    """
    if not isinstance(value, (int, float)):
        return value
    if value == 0:
        return 0
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 1: Temperature is ALWAYS integer
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit == "°C" or "temp" in param_key.lower():
        return int(round(value))
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 2: Decibels get max 1 decimal place
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit == "dB":
        return round(value, 1)
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 3: Percentages snap to standard values
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit == "%" or "tolerance" in param_key.lower():
        snapped = snap_to_standard_tolerance(abs(value))
        return snapped if value >= 0 else -snapped
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 4: E-Series snapping for resistors/capacitors
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if param_key in ["resistance", "capacitance", "inductance"]:
        # Determine E-series based on precision
        if precision_level == "high":
            series = "E96"
        elif precision_level == "adaptive":
            series = "E24"
        else:
            series = "E12"
        
        return snap_to_e_series(value, series)
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 5: ppm values - round to nearest 5 or 10
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit == "ppm/°C" or unit == "ppm/V":
        abs_val = abs(value)
        if abs_val >= 100:
            rounded = round(value / 10) * 10
        elif abs_val >= 10:
            rounded = round(value / 5) * 5
        else:
            rounded = round(value)
        return int(rounded)
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 6: Time values (ns, µs, ms)
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit in ["ns", "µs", "ms", "s"]:
        abs_val = abs(value)
        if abs_val >= 1000:
            return int(round(value))
        elif abs_val >= 100:
            return round(value, 1)
        elif abs_val >= 10:
            return round(value, 1)
        else:
            return round(value, 2)
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 7: Current and Voltage - aggressive rounding
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit in ["A", "mA", "µA", "V", "mV", "µV"]:
        abs_val = abs(value)
        if abs_val >= 1000:
            return int(round(value))
        elif abs_val >= 100:
            return int(round(value))
        elif abs_val >= 10:
            return round(value, 1)
        elif abs_val >= 1:
            return round(value, 2)
        else:
            return float(f"{value:.2g}")
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 8: Resistance - FIXED: Less aggressive rounding
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit in ["Ω", "mΩ", "kΩ", "MΩ"]:
        abs_val = abs(value)
        
        if abs_val >= 10000:
            return int(round(value))
        elif abs_val >= 1000:
            # Check if near E-series boundary
            nearest_e24 = snap_to_e_series(value, "E24")
            diff_pct = abs(value - nearest_e24) / nearest_e24 * 100
            
            if diff_pct < 2:  # Within 2% of E24 value
                return int(round(value))  # 1020 → 1000
            else:
                return round(value, 1)    # 1050 → 1050 or 1.05 kΩ
        elif abs_val >= 100:
            # CHANGED: Was int(round), now keeps 1 decimal
            return round(value, 1)  # 105.5 instead of 106
        elif abs_val >= 10:
            return round(value, 1)  # 10.5, not 10
        elif abs_val >= 1:
            return round(value, 2)  # 2.47
        else:
            return round(value, 3)  # 0.547
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 9: Capacitance - FIXED: Less aggressive rounding
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    if unit in ["pF", "nF", "µF", "mF"]:
        abs_val = abs(value)
        if abs_val >= 1000:
            return int(round(value))
        elif abs_val >= 100:
            return round(value, 1)  # CHANGED: Was int(round)
        elif abs_val >= 10:
            return round(value, 1)
        else:
            return round(value, 2)
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # RULE 10: Default adaptive rounding
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    abs_val = abs(value)
    
    if precision_level == "high":
        if abs_val >= 1000:
            return int(round(value))
        elif abs_val >= 100:
            return int(round(value))
        elif abs_val >= 10:
            return round(value, 2)
        else:
            return round(value, 3)
    
    elif precision_level == "minimal":
        if abs_val >= 10:
            return int(round(value))
        else:
            return round(value, 1)
    
    else:  # "adaptive"
        if abs_val >= 1000:
            return int(round(value))
        elif abs_val >= 100:
            return int(round(value))
        elif abs_val >= 10:
            return round(value, 1)
        elif abs_val >= 1:
            return round(value, 2)
        else:
            return float(f"{value:.2g}")

def validate_min_typ_max_spacing(min_val: Optional[float], 
                                 typ_val: Optional[float], 
                                 max_val: Optional[float],
                                 tolerance: float,
                                 unit: str = "") -> Tuple[Optional[float], Optional[float], Optional[float]]:
    """
    NEW FUNCTION v2.1: Verify that rounded values still preserve tolerance
    
    This prevents the rounding from destroying the min/typ/max relationship.
    Example: 10kΩ ±1% should give 9.9/10.0/10.1, not 9.9/10.0/10.0
    """
    # בדיקה שכל הערכים קיימים ומספריים
    if not all(isinstance(v, (int, float)) for v in [min_val, typ_val, max_val] if v is not None):
        return min_val, typ_val, max_val
    
    # אם חסר אחד מהערכים, אי אפשר לחשב מרווחים
    if min_val is None or typ_val is None or max_val is None:
        return min_val, typ_val, max_val
    
    if typ_val == 0:
        return min_val, typ_val, max_val

    # --- הקוד שביקשת לשלב (Low Side - טיפול במינימום) ---
    # הוספתי את אותו היגיון גם ל-Min כדי למנוע קריסה כלפי מעלה (9.99 -> 10.0)
    actual_tol_low = abs(typ_val - min_val) / typ_val
    if abs(actual_tol_low - tolerance) > 0.005:  # שיניתי מ-0.02 ל-0.005 כדי לתפוס 1%
        min_val = round(typ_val * (1 - tolerance), 2)

    # --- הקוד שביקשת לשלב (High Side - טיפול במקסימום) ---
    actual_tol_high = abs(max_val - typ_val) / typ_val
    
    # הסבר לתיקון: עבור 1%, השגיאה היא 0.01. התנאי חייב להיות קטן מ-0.01
    if abs(actual_tol_high - tolerance) > 0.005: 
        max_val = round(typ_val * (1 + tolerance), 2)
    
    # --- סוף הקוד המשולב ---

    # Special handling for resistance/capacitance in kΩ/µF range
    # זה נשאר כדי לוודא תצוגה יפה (למשל 10.0 במקום 10)
    if unit in ["kΩ", "µF", "nF"]:
        if isinstance(min_val, (int, float)) and min_val == int(min_val):
            min_val = round(min_val, 1)
        if isinstance(max_val, (int, float)) and max_val == int(max_val):
            max_val = round(max_val, 1)
    
    return min_val, typ_val, max_val

def select_best_display_unit(value: float, base_unit: str, possible_units: List[str]) -> str:
    """Select best unit for display based on value magnitude"""
    if not isinstance(value, (int, float)):
        return base_unit
    
    abs_val = abs(value)
    
    # Voltage
    if base_unit == "V":
        if abs_val < 0.1 and "mV" in possible_units:
            return "mV"
        if abs_val < 0.0001 and "µV" in possible_units:
            return "µV"
    
    # Current
    if base_unit == "A":
        if abs_val < 0.001 and "µA" in possible_units:
            return "µA"
        elif abs_val < 1 and "mA" in possible_units:
            return "mA"
    
    # Resistance
    if base_unit == "Ω":
        if abs_val >= 1e6 and "MΩ" in possible_units:
            return "MΩ"
        elif abs_val >= 1000 and "kΩ" in possible_units:
            return "kΩ"
        elif abs_val < 1 and "mΩ" in possible_units:
            return "mΩ"
    
    # Capacitance
    if base_unit in ["pF", "F"]:
        if abs_val >= 1e6 and "µF" in possible_units:
            return "µF"
        elif abs_val >= 1000 and "nF" in possible_units:
            return "nF"
        elif base_unit == "F" and abs_val < 1e-6 and "pF" in possible_units:
            return "pF"
    
    # Inductance
    if base_unit == "µH":
        if abs_val >= 1000 and "mH" in possible_units:
            return "mH"
        elif abs_val < 1 and "nH" in possible_units:
            return "nH"
    
    # Frequency
    if base_unit == "MHz":
        if abs_val >= 1000 and "GHz" in possible_units:
            return "GHz"
        elif abs_val < 1 and "kHz" in possible_units:
            return "kHz"
    
    return base_unit

def convert_for_display(value: Any, base_unit: str, target_unit: str, 
                       precision: str = "adaptive", param_key: str = "") -> Any:
    """Convert value to target unit and round for display"""
    if value == "-" or value is None:
        return "-"
    if not isinstance(value, (int, float)):
        return value
    
    # Get conversion factor
    factor = UNIT_CONVERSION.get((base_unit, target_unit), 1.0)
    converted = value * factor
    
    # Round the converted value
    rounded = apply_final_rounding(converted, precision, target_unit, param_key)
    
    return rounded

def validate_min_typ_max(min_val: Optional[float], typ_val: Optional[float], 
                        max_val: Optional[float]) -> Tuple[Optional[float], 
                                                            Optional[float], 
                                                            Optional[float]]:
    """Ensure min <= typ <= max constraint"""
    values = [v for v in [min_val, typ_val, max_val] if isinstance(v, (int, float))]
    
    if len(values) < 2:
        return min_val, typ_val, max_val
    
    values_sorted = sorted(values)
    
    result = [None, None, None]
    idx = 0
    if min_val is not None and isinstance(min_val, (int, float)):
        result[0] = values_sorted[idx]
        idx += 1
    if typ_val is not None and isinstance(typ_val, (int, float)):
        result[1] = values_sorted[idx]
        idx += 1
    if max_val is not None and isinstance(max_val, (int, float)):
        result[2] = values_sorted[idx]
    
    return tuple(result)

# ==========================================
# 4. MAIN DATASHEET GENERATOR CLASS
# ==========================================
class DatasheetGenerator:

    @staticmethod
    def should_display_tolerance(key: str, spec_type: str) -> bool:
        """
        ✅ FIX: Only show tolerance for passive nominal values
        """
        return (
            key in ["resistance", "capacitance", "inductance"] 
            and spec_type == "nominal"
        )
    def __init__(self, db: Dict[str, Any], enable_correlations: bool = True):
        self.db = db
        self.correlation_engine = PhysicalCorrelationEngine() if enable_correlations else None
        self.stored_params = {}  # For cross-parameter correlation
    
    def generate_single_sample(self, component_type: str, 
                               process_corner: Optional[str] = None,
                               rounding_strategy: str = "adaptive") -> Dict[str, Any]:
        """
        Generate a single synthetic datasheet sample with full physical correlation.
        
        Args:
            component_type: Type of component (MOSFET, CAPACITOR, etc.)
            process_corner: Optional process corner (TT/SS/FF), randomly chosen if None
            rounding_strategy: Rounding precision ("adaptive", "high", "minimal")
        
        Returns:
            Complete sample with metadata, ground_truth, and display_data
        """
        if component_type not in self.db:
            raise ValueError(f"Unknown component type: {component_type}")
        
        # 1. Generate unique ID
        sample_id = str(uuid.uuid4())
        
        comp_schema = self.db[component_type]
        
        # 2. Select archetype and process corner
        archetypes = comp_schema.get("archetypes", ["Standard"])
        target_archetype = random.choice(archetypes)
        
        if process_corner is None:
            process_corner = random.choice(list(PROCESS_CORNERS.keys()))
        corner_factors = PROCESS_CORNERS[process_corner]
        
        # 3. Select package
        target_package = self._select_package(comp_schema, target_archetype)
        
        # 4. Initialize storage
        ground_truth_json = {}
        display_tables = {}
        seen_labels = set()
        self.stored_params = {}  # Reset for each sample
        
        # 5. Process all parameters
        for section_name, params_list in comp_schema.items():
            if section_name == "archetypes":
                continue
            
            display_tables[section_name] = []
            
            for param_config in params_list:
                self._process_parameter(
                    param_config=param_config,
                    target_archetype=target_archetype,
                    target_package=target_package,
                    corner_factors=corner_factors,
                    ground_truth_json=ground_truth_json,
                    display_tables=display_tables,
                    section_name=section_name,
                    seen_labels=seen_labels,
                    rounding_strategy=rounding_strategy
                )
        
        # 6. Return complete sample
        return {
            "id": sample_id,
            "metadata": {
                "component_type": component_type,
                "archetype": target_archetype,
                "package": target_package,
                "process_corner": process_corner,
                "generation_timestamp": str(uuid.uuid1().time)
            },
            "display_metadata": {
                "rounding_strategy": rounding_strategy,
                "unit_scaling": "adaptive",
                "correlation_enabled": self.correlation_engine is not None
            },
            "ground_truth": ground_truth_json,
            "display_data": display_tables
        }
    
    def _select_package(self, comp_schema: Dict, target_archetype: str) -> str:
        """Select appropriate package for the component"""
        target_package = "Generic"
        
        if "PACKAGE" not in comp_schema:
            return target_package
        
        for p in comp_schema["PACKAGE"]:
            if p["key"] in ["package_code", "package_type"]:
                scenario = p["scenarios"][0]
                raw_limits = scenario["limits"]
                possible_packages = []
                
                if isinstance(raw_limits, dict):
                    possible_packages = raw_limits.get(target_archetype, [])
                    if not possible_packages and raw_limits:
                        possible_packages = list(raw_limits.values())[0]
                elif isinstance(raw_limits, list):
                    possible_packages = raw_limits
                
                if possible_packages:
                    target_package = random.choice(possible_packages)
                break
        
        return target_package
    
    def _process_parameter(self, param_config: Dict, target_archetype: str,
                           target_package: str, corner_factors: Dict,
                           ground_truth_json: Dict, display_tables: Dict,
                           section_name: str, seen_labels: set,
                           rounding_strategy: str):
        """
        ✅ COMPLETE FIXED VERSION
        """
        
        key = param_config["key"]
        std_unit = param_config["std_unit"]
        formal_name = param_config["llm_context"]["formal_name"]
        spec_type = param_config.get("spec_type", "nominal")
        
        tolerance = TOLERANCE_MAP.get(target_archetype, 0.05)
        
        filtered_scenarios = self._filter_scenarios(
            param_config["scenarios"], target_package
        )
        if not filtered_scenarios: 
            return
    
        num_to_select = random.randint(1, len(filtered_scenarios))
        if key in ["package_code", "package_type"]: 
            num_to_select = 1
        
        selected_scenarios = random.sample(filtered_scenarios, num_to_select)
        
        STRICT_SINGLE_VALUE = {
            "tolerance", "package_code", "package_type", "mounting_type", 
            "material", "core_material", "shielding"
        }
        
        for idx, scenario in enumerate(selected_scenarios):
            json_key = key if len(selected_scenarios) == 1 else f"{key}_{idx}"
            condition_str = scenario.get("condition", "N/A")
            
            # ✅ Step 1: Get industry-standard base value
            base_val = self._get_base_value(
                scenario["limits"], target_archetype, key, target_package
            )
            if base_val is None: 
                continue
            
            # Apply correlations
            if self.correlation_engine and isinstance(base_val, (int, float)):
                base_val = self._apply_correlations(key, base_val, target_package)
            
            self.stored_params[key] = base_val
            
            # ✅ Step 2: Calculate with exact tolerance
            if not isinstance(base_val, (int, float)):
                gt_min, gt_typ, gt_max = None, base_val, None
            else:
                gt_min, gt_typ, gt_max = self._calculate_min_typ_max(
                    base_val=base_val,
                    spec_type=spec_type,
                    target_archetype=target_archetype,
                    corner_factors=corner_factors,
                    condition_str=condition_str,
                    key=key
                )
            
            # ✅ Step 3: Round for ground_truth
            gt_min = apply_final_rounding(gt_min, "adaptive", std_unit, key) if gt_min else None
            gt_typ = apply_final_rounding(gt_typ, "adaptive", std_unit, key) if gt_typ else None
            gt_max = apply_final_rounding(gt_max, "adaptive", std_unit, key) if gt_max else None
            
            # Validate ordering
            if all(isinstance(v, (int, float)) for v in [gt_min, gt_typ, gt_max] if v):
                gt_min, gt_typ, gt_max = validate_min_typ_max(gt_min, gt_typ, gt_max)
            
            # Force single value
            if key in STRICT_SINGLE_VALUE:
                gt_min = None
                gt_max = None
                if gt_typ is None and gt_max is not None: 
                    gt_typ = gt_max
            
            # ✅ Step 4: Save to ground_truth
            ground_truth_json[json_key] = {
                "name": formal_name,
                "spec_type": spec_type,
                "condition": condition_str,
                "unit": std_unit,
                "min": gt_min,
                "typ": gt_typ,
                "max": gt_max
            }
            
            # ✅ Step 5: Create display from ground_truth
            self._prepare_display_data(
                param_config=param_config,
                key=key,
                condition_str=condition_str,
                gt_min=gt_min,
                gt_typ=gt_typ,
                gt_max=gt_max,
                std_unit=std_unit,
                display_tables=display_tables,
                section_name=section_name,
                seen_labels=seen_labels,
                tolerance=tolerance,
                spec_type=spec_type  # ✅ PASS THIS
            )

            
    def _filter_scenarios(self, scenarios: List[Dict], target_package: str) -> List[Dict]:
        """Filter scenarios based on package compatibility"""
        filtered = []
        for sc in scenarios:
            cond_text = sc["condition"]
            is_pkg_cond = any(pkg in cond_text for pkg in 
                            ["TO-", "SO-", "DPAK", "SOT", "SMA", "SMB", 
                             "DO-", "Axial", "Radial", "DFN", "QFN"])
            
            if is_pkg_cond:
                if target_package in cond_text:
                    filtered.append(sc)
            else:
                filtered.append(sc)
        
        return filtered if filtered else scenarios
    
    def _get_base_value(self, raw_limits: Any, target_archetype: str, 
                    key: str, target_package: str) -> Any:
        """
        ✅ FIXED: Return INDUSTRY-STANDARD values based on package/archetype
        """
        
        # Special handling for package-dependent parameters
        if key == "power_rating" or key == "power_dissipation":
            return POWER_RATINGS_BY_PACKAGE.get(target_package, 0.125)
        
        if key == "max_working_voltage":
            return VOLTAGE_RATINGS_BY_PACKAGE.get(target_package, 50)
        
        if key == "tcr" or key == "temperature_coefficient":
            return TCR_BY_ARCHETYPE.get(target_archetype, 100)
        
        if key == "noise" or key == "current_noise":
            typ, max_val = NOISE_LEVELS.get(target_archetype, (-40, -20))
            return max_val  # Return Max, we'll calculate Typ later
        
        # For other parameters, use original logic
        actual_values_list = []
        
        if isinstance(raw_limits, dict):
            actual_values_list = raw_limits.get(target_archetype)
            if actual_values_list is None and raw_limits:
                actual_values_list = list(raw_limits.values())[0]
        else:
            actual_values_list = raw_limits
        
        if not actual_values_list:
            return None
        
        if key in ["package_code", "package_type"]:
            return target_package
        
        if not all(isinstance(v, (int, float)) for v in actual_values_list):
            return random.choice(actual_values_list)
        
        # Intelligent selection based on archetype
        sorted_values = sorted(actual_values_list)
        is_precision = "Precision" in target_archetype
        
        lower_is_better = [
            "tcr", "temperature_coefficient", "voltage_coefficient",
            "noise", "offset_voltage", "esr", "rds_on", "thermal_resistance"
        ]
        
        if key in lower_is_better:
            return sorted_values[0] if is_precision else sorted_values[len(sorted_values)//2]
        else:
            return random.choice(actual_values_list)
        
    def _apply_correlations(self, key: str, base_val: float, 
                           target_package: str) -> float:
        """Apply physical correlations between parameters"""
        if not self.correlation_engine:
            return base_val
        
        # MOSFET correlations
        if key == "rds_on" and "vdss" in self.stored_params:
            vds = self.stored_params["vdss"]
            return self.correlation_engine.adjust_rds_based_on_vds(vds, base_val)
        
        # Diode correlations
        if key == "forward_current":
            return self.correlation_engine.adjust_forward_current_based_on_package(
                target_package, base_val
            )
        
        # Capacitor correlations
        if key == "esr" and "capacitance" in self.stored_params:
            cap = self.stored_params["capacitance"]
            return self.correlation_engine.adjust_esr_based_on_capacitance(cap, base_val)
        
        # Power dissipation correlation
        if key == "power_dissipation":
            return self.correlation_engine.adjust_power_dissipation_based_on_package(
                target_package, base_val
            )
        
        return base_val
    
    def _calculate_min_typ_max(self, base_val: Any, spec_type: str, 
                          target_archetype: str, corner_factors: Dict,
                          condition_str: str, key: str) -> Tuple:
        """
        ✅ FIXED: Calculate with EXACT tolerance preservation
        """
        
        if base_val is None:
            return None, None, None
        
        if not isinstance(base_val, (int, float)):
            return None, base_val, None
        
        # Get tolerance from archetype
        tolerance = TOLERANCE_MAP.get(target_archetype, 0.05)
        
        # Apply temperature derating
        temp = extract_temperature_from_condition(condition_str)
        if temp and temp > 25:
            base_val = apply_temperature_derating(base_val, temp)
        
        # ✅ FIX: E-series snap BEFORE tolerance calculation
        if key in ["resistance", "capacitance", "inductance"]:
            series = "E96" if tolerance <= 0.01 else "E24"
            base_val = snap_to_e_series(base_val, series)
        
        # Check if passive (ignore process corners)
        is_passive = any(p in target_archetype for p in 
                        ["Resistor", "Capacitor", "Inductor", "Standard_SMD", 
                         "Precision_ThinFilm", "Ceramic_MLCC", "Electrolytic"])
        
        if is_passive:
            corner_factors = {"min_factor": 1.0, "typ_factor": 1.0, "max_factor": 1.0}
        
        # Calculate based on spec_type
        if spec_type == "max_rating":
            return None, None, base_val
        
        elif spec_type == "max_limit":
            # ✅ FIX: For noise, Typ < Max (lower dB = better)
            if key in ["noise", "current_noise"]:
                raw_max = base_val  # This is the worst case
                raw_typ = base_val - 10  # 10dB better than max
                return None, raw_typ, raw_max
            
            # For other max_limit parameters
            raw_max = base_val
            raw_typ = base_val * random.uniform(0.50, 0.70)  # ✅ FIX: Lower typical
            return None, raw_typ, raw_max
        
        elif spec_type == "min_limit":
            raw_min = base_val
            raw_typ = base_val * random.uniform(1.20, 1.50)
            return raw_min, raw_typ, None
        
        elif spec_type == "nominal":
            # ✅ FIX: EXACT tolerance calculation
            raw_typ = base_val
            raw_min = raw_typ * (1 - tolerance)
            raw_max = raw_typ * (1 + tolerance)
            
            return raw_min, raw_typ, raw_max
        
        else:
            return None, base_val, None

    
    
    def _prepare_display_data(
            self,
        param_config: Dict,
        key: str,
        condition_str: str,
        gt_min: Any,
        gt_typ: Any,
        gt_max: Any,
        std_unit: str,
        display_tables: Dict,
        section_name: str,
        seen_labels: set,
        tolerance: Optional[float] = None,
        spec_type: str = "nominal"  # ✅ ADD THIS
    ):
        """
        ✅ FIXED: Display only with correct tolerance placement
        """
        
        SINGLE_VALUE_PARAMS = {
            "tolerance", "package_code", "package_type", 
            "mounting_type", "material", "core_material"
        }
        
        should_show_single = key in SINGLE_VALUE_PARAMS
        
        # Determine best display unit
        val_check = gt_typ if isinstance(gt_typ, (int, float)) else gt_max
        display_unit = std_unit
        
        if isinstance(val_check, (int, float)):
            possible_units = [pair[1] for pair in UNIT_CONVERSION.keys() 
                             if pair[0] == std_unit]
            display_unit = select_best_display_unit(val_check, std_unit, possible_units)
        
        # Convert to display
        def gt_to_display(gt_value):
            if gt_value is None or gt_value == "—":
                return "—"
            if not isinstance(gt_value, (int, float)):
                return gt_value
            
            factor = UNIT_CONVERSION.get((std_unit, display_unit), 1.0)
            converted = gt_value * factor
            
            # Minimal rounding for display
            if display_unit in ["kΩ", "MΩ", "µF", "nF"]:
                if converted >= 100:
                    return round(converted, 1)
                else:
                    return round(converted, 2)
            
            return converted
        
        display_min = gt_to_display(gt_min)
        display_typ = gt_to_display(gt_typ)
        display_max = gt_to_display(gt_max)
        
        # Build row
        if should_show_single:
            row = {
                "Parameter": param_config.get('label', key),
                "Value": display_typ,
                "Units": display_unit,
                "Conditions": condition_str
            }
        else:
            row = {
                "Parameter": param_config.get('label', key),
                "Min": display_min,
                "Typ": display_typ,
                "Max": display_max,
                "Units": display_unit,
                "Conditions": condition_str
            }
        
        # ✅ FIX: Only add tolerance for passive nominal values
        if tolerance is not None and self.should_display_tolerance(key, spec_type):
            row["Tolerance"] = f"±{tolerance*100:.1f}%"
        
        # Add to table
        if section_name not in display_tables:
            display_tables[section_name] = []
        
        label_key = (section_name, row["Parameter"])
        if label_key not in seen_labels:
            seen_labels.add(label_key)
            display_tables[section_name].append(row)
        
        

# ==========================================
# 5. BATCH GENERATION & UTILITIES
# ==========================================
def generate_batch(db: Dict, component_type: str, num_samples: int = 100,
                  process_corners: Optional[List[str]] = None) -> List[Dict]:
    """Generate a batch of samples with diversity"""
    gen = DatasheetGenerator(db, enable_correlations=True)
    samples = []
    
    if process_corners is None:
        process_corners = list(PROCESS_CORNERS.keys())
    
    for i in range(num_samples):
        corner = random.choice(process_corners)
        rounding = random.choice(["adaptive", "high", "minimal"])
        
        try:
            sample = gen.generate_single_sample(
                component_type=component_type,
                process_corner=corner,
                rounding_strategy=rounding
            )
            samples.append(sample)
            
            if (i + 1) % 10 == 0:
                print(f"Generated {i + 1}/{num_samples} samples...")
        
        except Exception as e:
            print(f"Error generating sample {i + 1}: {str(e)}")
            continue
    
    return samples

def save_samples_to_json(samples: List[Dict], filename: str):
    """Save generated samples to JSON file"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(samples, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved {len(samples)} samples to {filename}")

def print_sample_summary(sample: Dict):
    """Print sample as formatted JSON"""
    print(json.dumps(sample, indent=2, ensure_ascii=False))

def validate_sample_integrity(sample: Dict) -> Dict[str, Any]:
    """Validate sample for physical consistency and data quality"""
    issues = []
    warnings = []
    
    # Check 1: All ground truth values have corresponding display values
    gt_keys = set(sample['ground_truth'].keys())
    display_keys = set()
    for section in sample['display_data'].values():
        for row in section:
            display_keys.add(row['key'])
    
    missing_in_display = gt_keys - display_keys
    if missing_in_display:
        warnings.append(f"GT keys missing in display: {missing_in_display}")
    
    # Check 2: Min <= Typ <= Max in ground truth
    for key, data in sample['ground_truth'].items():
        min_val = data.get('min')
        typ_val = data.get('typ')
        max_val = data.get('max')
        
        values = [v for v in [min_val, typ_val, max_val] if isinstance(v, (int, float))]
        if len(values) >= 2 and values != sorted(values):
            issues.append(f"❌ {key}: Min/Typ/Max ordering violated: {min_val}/{typ_val}/{max_val}")
    
    # Check 3: Tolerance preservation
    for key, data in sample['ground_truth'].items():
        if key in ["resistance", "capacitance", "inductance"]:
            min_val = data.get('min')
            typ_val = data.get('typ')
            max_val = data.get('max')
            
            if all(isinstance(v, (int, float)) for v in [min_val, typ_val, max_val]):
                tolerance_low = abs(typ_val - min_val) / typ_val if typ_val else 0
                tolerance_high = abs(max_val - typ_val) / typ_val if typ_val else 0
                
                if abs(tolerance_low - tolerance_high) > 0.02:
                    warnings.append(f"⚠️  {key}: Asymmetric tolerance {tolerance_low:.1%}/{tolerance_high:.1%}")
    
    # --- FIX 2: Return dictionary correctly (same line or strict indentation) ---
    return {
        "valid": len(issues) == 0,
        "issues": issues,
        "warnings": warnings,
        "score": 100 - (len(issues) * 10) - (len(warnings) * 2)
    }
# ==========================================
# 8. MAIN EXECUTION
# ==========================================

if __name__ == "__main__":
    print("🔄 Checking prerequisites...")

    # 1. בדיקה שהמילון קיים בזיכרון
    if 'UNIFIED_COMPONENT_DB' not in globals():
        print("❌ Error: 'UNIFIED_COMPONENT_DB' is missing!")
        print("   👉 Solution: Run the cell that defines the component dictionary first.")
    
    else:
        print("✅ Database found. Starting generation...")
        print("\n" + "="*80)
        print(" 🏭 GENERATING PRODUCTION BATCH")
        print("="*80)
        
        PRODUCTION_SAMPLES = []
        
        # רשימת הרכיבים ליצירה
        components_to_generate = ["RESISTOR", "CAPACITOR", "DIODE", "MOSFET", "VOLTAGE_REGULATOR"]
        
        for comp_type in components_to_generate:
            # בדיקה שהרכיב קיים במילון לפני שמתחילים
            if comp_type not in UNIFIED_COMPONENT_DB:
                print(f"⚠️ Warning: {comp_type} not found in DB. Skipping.")
                continue

            print(f"\n📦 Generating 20 samples for {comp_type}...")
            
            try:
                # קריאה לפונקציית הג'ינרוט
                batch = generate_batch(
                    UNIFIED_COMPONENT_DB, 
                    comp_type, 
                    num_samples=20
                )
                PRODUCTION_SAMPLES.extend(batch)
                print(f"   ---> Added {len(batch)} samples.")
                
            except Exception as e:
                print(f"   ❌ Critical Error generating {comp_type}: {e}")

        # סיכום ושמירה
        print("\n" + "="*80)
        print(f"✅ COMPLETE: Generated {len(PRODUCTION_SAMPLES)} total samples.")
        print("="*80)
        
        if len(PRODUCTION_SAMPLES) > 0:
            # --- שמירה לקובץ (החלק שהוספת) ---
            master_filename = "all_datasheets_db.json"
            try:
                save_samples_to_json(PRODUCTION_SAMPLES, master_filename)
                print(f"\n💾 SUCCESS: Database saved to '{master_filename}'")
                print(f"   You can now search for IDs in this file.")
                
                # הדפסת דוגמה אחת כדי שתראה שזה עובד
                print("\n📄 Random Sample Preview:")
                print_sample_summary(PRODUCTION_SAMPLES[0])
                
            except Exception as e:
                print(f"❌ Error saving file: {e}")
        else:
            print("⚠️ No samples were generated. Check the errors above.")

🔄 Checking prerequisites...
✅ Database found. Starting generation...

 🏭 GENERATING PRODUCTION BATCH

📦 Generating 20 samples for RESISTOR...
Generated 10/20 samples...
Generated 20/20 samples...
   ---> Added 20 samples.

📦 Generating 20 samples for CAPACITOR...
Generated 10/20 samples...
Generated 20/20 samples...
   ---> Added 20 samples.

📦 Generating 20 samples for DIODE...
Generated 10/20 samples...
Generated 20/20 samples...
   ---> Added 20 samples.

📦 Generating 20 samples for MOSFET...
Generated 10/20 samples...
Generated 20/20 samples...
   ---> Added 20 samples.

📦 Generating 20 samples for VOLTAGE_REGULATOR...
Generated 10/20 samples...
Generated 20/20 samples...
   ---> Added 20 samples.

✅ COMPLETE: Generated 100 total samples.
✅ Saved 100 samples to all_datasheets_db.json

💾 SUCCESS: Database saved to 'all_datasheets_db.json'
   You can now search for IDs in this file.

📄 Random Sample Preview:
{
  "id": "7e49b55d-349f-458c-a975-689780335fd2",
  "metadata": {
    "compo

# Creating HTML

In [65]:
TABLE_STRUCTURES = {
    "standard_full": {
        "name": "Standard Full",
        "columns": ["Parameter", "Symbol", "Condition", "Min", "Typ", "Max", "Unit"],
        "merge_limits": False,     # משאיר Min/Typ/Max נפרדים
        "split_sections": True,    # מפצל לטבלאות לפי נושאים (חשמלי, תרמי וכו')
        "show_symbol": True
    },
    "compact_commercial": {
        "name": "Compact Commercial",
        "columns": ["Parameter", "Condition", "Value", "Unit"],
        "merge_limits": "typ_preferred", # מציג Typ כ-Value. אם אין, מציג Max.
        "split_sections": True,
        "show_symbol": False
    },
    "minimalist_list": {
        "name": "Minimalist List",
        "columns": ["Parameter", "Value", "Unit"],
        "merge_limits": "range",   # מציג טווח "Min-Max" בתוך עמודת Value
        "split_sections": False,   # מאחד הכל לטבלה אחת ארוכה
        "show_symbol": False
    },
    "scientific_detailed": {
        "name": "Scientific Detailed",
        "columns": ["Parameter", "Symbol", "Test Conditions", "Min", "Max", "Unit"],
        "merge_limits": False,
        "split_sections": True,
        "show_symbol": True,
        "skip_typ": True           # מדלג על עמודת Typ (נפוץ ברכיבי בטיחות/קיצון)
    },
    "legacy_databook": {
        "name": "Legacy Databook",
        "columns": ["Characteristic", "Symbol", "Min", "Typ", "Max", "Unit"],
        "merge_limits": False,
        "split_sections": True,
        "show_symbol": True,
        "condition_in_name": True  # משרשר את ה-Condition לשם הפרמטר
    }
}

VISUAL_STYLES = {
    "classic_standard": {
        "name": "Classic Standard (TI Style)",
        "css": """
            body { font-family: Arial, Helvetica, sans-serif; color: #333; line-height: 1.4; }
            h1 { color: #003366; border-bottom: 2px solid #003366; }
            table { width: 100%; border-collapse: collapse; margin-bottom: 20px; font-size: 13px; }
            th { background-color: #003366; color: white; text-align: left; padding: 8px; }
            td { border-bottom: 1px solid #ddd; padding: 6px; }
            tr:nth-child(even) { background-color: #f2f2f2; }
        """
    },
    "vintage_databook": {
        "name": "Vintage Databook (80s Style)",
        "css": """
            body { font-family: 'Courier New', Courier, monospace; color: #000; }
            h1 { text-transform: uppercase; text-decoration: underline; }
            table { width: 100%; border: 2px solid #000; border-collapse: collapse; margin-bottom: 25px; }
            th { border-bottom: 2px solid #000; border-right: 1px solid #000; padding: 5px; text-transform: uppercase; }
            td { border-right: 1px solid #000; border-bottom: 1px solid #000; padding: 5px; }
        """
    },
    "modern_tech": {
        "name": "Modern Tech (IoT Style)",
        "css": """
            body { font-family: 'Segoe UI', Roboto, sans-serif; color: #444; background: #fcfcfc; }
            h1 { color: #00BCD4; font-weight: 300; letter-spacing: 1px; }
            table { width: 100%; border-collapse: separate; border-spacing: 0; box-shadow: 0 2px 15px rgba(0,0,0,0.05); border-radius: 8px; overflow: hidden; }
            th { background-color: #f8f9fa; color: #555; font-weight: 600; padding: 12px; border-bottom: 2px solid #00BCD4; }
            td { padding: 12px; border-bottom: 1px solid #eee; }
        """
    },
    "industrial_heavy": {
        "name": "Industrial Heavy",
        "css": """
            body { font-family: Verdana, sans-serif; color: #222; }
            h1 { background-color: #FF5722; color: white; padding: 10px; font-family: Impact, sans-serif; }
            table { width: 100%; border: 3px solid #444; border-collapse: collapse; }
            th { background-color: #444; color: #FF5722; padding: 10px; border: 1px solid #666; }
            td { border: 1px solid #999; padding: 8px; font-weight: bold; }
        """
    },
    "scientific_precision": {
        "name": "Scientific Precision",
        "css": """
            body { font-family: 'Times New Roman', Times, serif; color: #111; }
            h1 { font-style: italic; border-bottom: 1px solid #000; }
            table { width: 100%; border-top: 2px solid #000; border-bottom: 2px solid #000; border-collapse: collapse; }
            th { border-bottom: 1px solid #000; padding: 4px; font-style: italic; }
            td { padding: 4px; border-right: 1px dotted #ccc; }
            td:last-child { border-right: none; }
        """
    },
    "compact_japanese": {
        "name": "Compact Japanese",
        "css": """
            body { font-family: Tahoma, sans-serif; font-size: 11px; color: #000033; }
            h1 { font-size: 16px; color: #000080; border-left: 5px solid #000080; padding-left: 10px; }
            table { width: 100%; border-collapse: collapse; border: 1px solid #999; }
            th { background-color: #e6e6ff; color: #000080; padding: 3px; border: 1px solid #999; font-size: 10px; }
            td { border: 1px solid #ccc; padding: 2px 4px; }
        """
    },
    "euro_minimalist": {
        "name": "Euro Minimalist",
        "css": """
            body { font-family: 'Helvetica Neue', Helvetica, sans-serif; color: #2c3e50; }
            h1 { color: #c0392b; font-size: 24px; margin-bottom: 30px; }
            table { width: 100%; border-collapse: collapse; }
            th { text-align: left; padding: 15px 5px; border-bottom: 2px solid #c0392b; color: #7f8c8d; font-weight: normal; }
            td { padding: 10px 5px; border-bottom: 1px solid #ecf0f1; }
        """
    },
    "high_contrast_print": {
        "name": "High Contrast (Print)",
        "css": """
            body { font-family: Arial, sans-serif; color: #000; background: #fff; }
            h1 { background: #000; color: #fff; padding: 5px 10px; }
            table { width: 100%; border: 4px solid #000; border-collapse: collapse; }
            th { background: #000; color: #fff; padding: 10px; font-weight: bold; border: 1px solid #fff; }
            td { border: 1px solid #000; padding: 8px; font-weight: bold; }
        """
    },
    "dark_mode_dev": {
        "name": "Dark Mode (Developer)",
        "css": """
            body { font-family: 'Consolas', 'Monaco', monospace; background-color: #1e1e1e; color: #d4d4d4; }
            h1 { color: #569cd6; }
            table { width: 100%; border-collapse: collapse; }
            th { color: #4ec9b0; text-align: left; padding: 10px; border-bottom: 1px solid #333; }
            td { padding: 10px; border-bottom: 1px solid #2d2d2d; color: #ce9178; }
            tr:hover { background-color: #2a2d2e; }
        """
    },
    "eco_green": {
        "name": "Eco Green",
        "css": """
            body { font-family: 'Trebuchet MS', sans-serif; color: #2e7d32; }
            h1 { color: #1b5e20; }
            table { width: 100%; border-collapse: separate; border-spacing: 0; border: 1px solid #a5d6a7; border-radius: 10px; }
            th { background-color: #e8f5e9; color: #1b5e20; padding: 10px; first-child: border-top-left-radius: 10px; }
            td { padding: 8px; border-top: 1px solid #a5d6a7; }
        """
    }
}
SECTION_TITLES = {
    "ABS_MAX": "Absolute Maximum Ratings",
    "ELEC_CHAR": "Electrical Characteristics",
    "PACKAGE": "Mechanical Information",
    "THERMAL": "Thermal Characteristics"
}

In [66]:
import requests
import json
import re
import random
from typing import Dict

# ==========================================
# CONSTANTS
# ==========================================
OLLAMA_URL = "http://localhost:11434/api/generate"

# ==========================================
# 1. HELPER: OLLAMA COMMUNICATION
# ==========================================
def query_ollama_fixed(prompt: str, debug: bool = False) -> str:
    """
    שולח פרומפט ל-Ollama ומחזיר את הטקסט הגולמי.
    כולל טיפול בשגיאות חיבור.
    """
    try:
        if debug:
            print(f"     [DEBUG] Sending to Ollama ({len(prompt)} chars)...")

        payload = {
            "model": MODEL_TO_USE,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.3,  # טמפרטורה נמוכה לקבלת JSON יציב
                "num_predict": 1024,
                "top_p": 0.9
            }
        }

        response = requests.post(OLLAMA_URL, json=payload, timeout=60)
        
        # בדיקת סטטוס HTTP
        if response.status_code != 200:
            return f"Error: Ollama returned status {response.status_code}"

        result = response.json()
        
        # חילוץ הטקסט מתוך התשובה
        output = result.get('response', '')
        return output

    except requests.exceptions.RequestException as e:
        print(f"     [ERROR] Connection to Ollama failed: {e}")
        return "Error: Connection failed"
    except Exception as e:
        print(f"     [ERROR] General error: {e}")
        return f"Error: {str(e)}"

# ==========================================
# 2. MAIN FUNCTION: CREATIVE METADATA
# ==========================================
def generate_creative_metadata(sample: Dict) -> Dict:
    """
    בונה פרומפט, שולחת ל-Ollama, מנקה את התשובה ומחזירה אובייקט JSON.
    """
    
    # 1. חילוץ הקשר מתוך ה-sample
    meta = sample.get('metadata', {})
    comp_type = meta.get('component_type', 'Component')
    archetype = meta.get('archetype', 'Standard')
    package = meta.get('package', 'Generic')

    # 2. בניית הפרומפט (ההוראות ל-LLM)
    prompt = f"""
    You are a technical datasheet writer.
    
    Product Info:
    - Component Type: {comp_type}
    - Technology: {archetype}
    - Package: {package}

    Task: Generate a VALID JSON object with creative marketing details.
    
    Required JSON Structure:
    {{
        "marketing_name": "An industrial model number (e.g. XF-2000-PRO)",
        "manufacturer": "A fictional professional company name",
        "description": "A professional 2-sentence technical description highlighting benefits.",
        "section_intros": {{
            "ABS_MAX": "One sentence explaining absolute maximum ratings.",
            "ELEC_CHAR": "One sentence explaining electrical characteristics."
        }},
        "footnotes": [
            "Technical note 1 (e.g. regarding pulse testing)",
            "Technical note 2 (e.g. regarding thermal conditions)"
        ]
    }}

    CRITICAL: Output ONLY the raw JSON string. Do NOT use markdown code blocks (```json).
    """

    # 3. קריאה ל-Ollama (באמצעות הפונקציה הפנימית/החיצונית)
    # אנחנו משתמשים בפונקציה שהגדרנו למעלה
    raw_response = query_ollama_fixed(prompt, debug=False)

    # 4. ניקוי ופירסור (Parsing)
    # לעיתים המודל מחזיר טקסט לפני ה-JSON או שגיאות רשת
    if "Error:" in raw_response:
        print(f"   ⚠️ Ollama call failed. Using fallback data.")
        return _get_fallback_metadata(comp_type, archetype, package)

    try:
        # ניקוי: הסרת תווי Markdown אם קיימים
        clean_json = raw_response.strip()
        if "```" in clean_json:
            # מנסה לחלץ רק את מה שבין הסוגריים המסולסלים החיצוניים
            match = re.search(r'(\{.*\})', clean_json, re.DOTALL)
            if match:
                clean_json = match.group(1)
            else:
                # ניקוי גס של ```json ו-```
                clean_json = clean_json.replace("```json", "").replace("```", "")

        # המרה למילון פייתון
        data = json.loads(clean_json)
        return data

    except json.JSONDecodeError:
        print(f"   ⚠️ Failed to decode JSON from Ollama. Raw: {raw_response[:50]}...")
        return _get_fallback_metadata(comp_type, archetype, package)

# ==========================================
# 3. FALLBACK (מנגנון הגנה)
# ==========================================
def _get_fallback_metadata(comp_type, archetype, package):
    """מחזיר נתונים גנריים במקרה של כישלון"""
    return {
        "marketing_name": f"GEN-{comp_type[:3].upper()}-{random.randint(1000,9999)}",
        "manufacturer": "Standard Semi Corp",
        "description": f"Standard {comp_type} utilizing {archetype} technology. Optimized for general purpose applications in {package} package.",
        "section_intros": {
            "ABS_MAX": "Stresses beyond those listed may cause permanent damage.",
            "ELEC_CHAR": "Parameters guaranteed at 25°C unless otherwise noted."
        },
        "footnotes": [
            "Specifications subject to change without notice.",
            "Pulse width limited by maximum junction temperature."
        ]
    }
# ==========================================
# פונקציה 2: המהנדס (Python HTML Renderer)
# ==========================================
def render_html_table(data_rows: list, structure_key: str) -> str:
    """
    בונה HTML של טבלה על בסיס הנתונים ומפתח המבנה.
    """
    # שליפת הגדרות המבנה
    structure = TABLE_STRUCTURES.get(structure_key, TABLE_STRUCTURES["standard_full"])
    columns_config = structure["columns"]
    merge_values = structure.get("merge_values", False)

    html = "<table>\n<thead>\n<tr>"

    # 1. יצירת כותרות
    for col in columns_config:
        html += f"<th>{col}</th>"
    html += "</tr>\n</thead>\n<tbody>"

    # 2. יצירת שורות
    for row in data_rows:
        html += "<tr>"
        
        for col_name in columns_config:
            cell_val = "—"
            
            # --- לוגיקה למיפוי עמודות ---
            
            # שם הפרמטר
            if col_name in ["Parameter", "Characteristic"]:
                cell_val = row.get("label", "—")
                # הוספת כוכבית להערות אם צריך (לפי דרישה 7 - נממש בהמשך פשוט כטקסט כרגע)
            
            # סימול (Symbol)
            elif col_name == "Symbol":
    # בדיקה: האם קיים סימול אמיתי שהגיע מהדאטה?
                if row.get("symbol"):
                    cell_val = row.get("symbol")
                else:
                    # Fallback: אם אין סימול, מייצרים ראשי תיבות (כמו שהיה קודם)
                    label = row.get("label", "")
                    cell_val = "".join([w[0] for w in label.split()]).upper() if label else "-"

            # תנאי בדיקה
            elif col_name in ["Condition", "Test Conditions"]:
                cell_val = row.get("condition", "—")

            # יחידות
            elif col_name == "Unit":
                cell_val = row.get("unit", "")

            # ערכים נפרדים (Min, Typ, Max)
            elif col_name in ["Min", "Typ", "Max"]:
                key = col_name.lower() # המרה לאות קטנה כי ב-JSON זה min/typ/max
                val = row.get(key)
                cell_val = str(val) if val is not None else "—"

            # ערך מאוחד (Value) - הלוגיקה המרכזית שלך
            elif col_name == "Value":
                min_v = row.get("min")
                typ_v = row.get("typ")
                max_v = row.get("max")
                
                # סדר עדיפויות: Typ -> Max -> Min
                if typ_v is not None:
                    cell_val = str(typ_v)
                elif max_v is not None:
                    cell_val = str(max_v) # במקרים של ABS MAX לפעמים יש רק Max
                elif min_v is not None:
                    cell_val = str(min_v)
                
                # אם רוצים להציג טווח
                if min_v is not None and max_v is not None and merge_values == "range":
                     cell_val = f"{min_v} ... {max_v}"

            html += f"<td>{cell_val}</td>"
        
        html += "</tr>\n"

    html += "</tbody>\n</table>"
    return html


# ==========================================
# פונקציה 3: המנהל (Assembler)
# ==========================================
def assemble_final_datasheet(sample: dict, creative_data: dict) -> str:
    """
    מרכיבה את הדף הסופי ומוסיפה את ה-ID.
    """
    # 1. בחירה רנדומלית של סגנון ומבנה
    style_key = random.choice(list(VISUAL_STYLES.keys()))
    structure_key = random.choice(list(TABLE_STRUCTURES.keys()))
    style_data = VISUAL_STYLES[style_key]
    
    # 2. חילוץ מידע
    # ### שינוי 1: חילוץ ה-ID מתוך ה-sample ###
    dataset_id = sample.get('id', 'UNKNOWN_ID')
    
    marketing_name = creative_data.get("marketing_name", "Component")
    manufacturer = creative_data.get("manufacturer", "Generic Corp")
    description = creative_data.get("description", "")
    section_intros = creative_data.get("section_intros", {})
    footnotes = creative_data.get("footnotes", [])

    # 3. בניית ה-HTML
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>{marketing_name} Datasheet</title>
        <style>
            {style_data['css']}
            /* ### שינוי 2: הוספת עיצוב קטן ל-ID בצד ### */
            .meta-id {{
                position: absolute;
                top: 10px;
                right: 10px;
                font-family: monospace;
                font-size: 10px;
                color: #aaa;
            }}
            .datasheet-header {{ position: relative; }}
        </style>
    </head>
    <body>
        <div class="datasheet-header">
            <div class="meta-id">Ref ID: {dataset_id}</div>
            
            <small>{manufacturer}</small>
            <h1>{marketing_name}</h1>
            <p><strong>Description:</strong> {description}</p>
        </div>
    """

    # ... (המשך הקוד נשאר אותו דבר: לולאה על הסקשנים והטבלאות) ...
    
    display_data = sample.get('display_data', {})
    sections_order = ["ABS_MAX", "THERMAL", "ELEC_CHAR", "PACKAGE"]
    
    for section_key in sections_order:
        if section_key not in display_data or not display_data[section_key]:
            continue
        rows = display_data[section_key]
        nice_title = SECTION_TITLES.get(section_key, section_key)
        intro_text = section_intros.get(section_key, "")
        
        table_html = render_html_table(rows, structure_key)

        html += f"""
        <div class="section">
            <h2 class="section-title">{nice_title}</h2>
            <p><em>{intro_text}</em></p>
            {table_html}
        </div>
        """

    if footnotes:
        html += "<div class='footer-notes'><h3>Notes:</h3><ul>"
        for i, note in enumerate(footnotes):
            html += f"<li><span class='note-marker'>*</span> {note}</li>"
        html += "</ul></div>"

    html += """
    </body>
    </html>
    """
    
    return html

In [ ]:

# יצירת תיקייה לשמירה
output_dir = "final_datasheets"
os.makedirs(output_dir, exist_ok=True)

print(f"🚀 Processing {len(PRODUCTION_SAMPLES)} samples...")

for i, sample in enumerate(PRODUCTION_SAMPLES):
    try:
        # שלב 1: הקופירייטר (Ollama)
        # בפועל זה ייקח זמן בגלל קריאת הרשת, כרגע זה מהיר בגלל ה-Mock
        creative_content = generate_creative_metadata(sample)
        
        # שלב 2 + 3: המהנדס והמנהל מייצרים את ה-HTML
        final_html = assemble_final_datasheet(sample, creative_content)
        
        # שמירה לקובץ
        filename = f"{output_dir}/ds_{i}_{sample['metadata']['component_type']}.html"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(final_html)
            
        print(f"  ✅ Generated: {filename}")
        
    except Exception as e:
        print(f"  ❌ Error processing sample {i}: {e}")

print("Done.")

🚀 Processing 100 samples...


# Find with ID

In [78]:
target_id = "de83f22b-90bb-45a8-b383-2d2b0cb690d2"

def get_component_by_id(target_id, filename="ground_truth.json"): # שים לב לשם הקובץ
    print(f"🔎 Searching for ID: {target_id}...")
    
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        for sample in data:
            if sample['id'] == target_id:
                print("✅ Found!")
                return sample
                
        print("❌ ID not found.")
        return None
        
    except FileNotFoundError:
        print(f"❌ File '{filename}' not found. Run the generator first.")
        return None

# --- הקריאה לפונקציה והדפסה ---

# 1. קריאה לפונקציה ושמירת התוצאה במשתנה
found_obj = get_component_by_id(target_id, filename="ground_truth.json")

# 2. הדפסה מפורשת של האובייקט אם נמצא
if found_obj:
    print("\n📦 Object Content:")
    print(json.dumps(found_obj, indent=2, ensure_ascii=False))

🔎 Searching for ID: de83f22b-90bb-45a8-b383-2d2b0cb690d2...
❌ File 'ground_truth.json' not found. Run the generator first.
